# 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算するプログラム

- 全体の制御モジュール
  - 一次元壁体の構成＋熱物性の入力
  - 二組の角周波数の選択
  - 複数選択時の単位応答の選り抜き
- 一次元壁体の構成＋熱物性から周波数応答を算出するモジュール
- 二組の角周波数に対応した周波数応答から指数項二項で近似した単位応答を算出するモジュール





## 1. 全体制御モジュール



### 1.1 入力値と出力値

#### (1) 一次元躯体の設定方法

- 一次元躯体$m$($m=1$～$M$)について以下の設定をする。、

- 躯体$m$の表面積$ S_m \; [m^2]$

- 躯体$m$の層構成を、全体で$N_m$層($N_m-2\;$層の躯体構成要素＋その両面の表面熱伝達層)として設定する。

  - 層$1 \; (n_m=1)$：単位応答を計算する側(例：外壁の室内側)の表面熱伝達層
  - 層$2$～層$N_m-1 \; (n_m=2$～$N_m-1)$：躯体を構成する部材の層(中空層含む)
  - 層$N_m \; (n_m=N_m)$：反対側(例：外壁の外気側)の表面熱伝達層
  - 「室内側表面熱伝達層を含まない」周波数応答・単位応答を求める際でも、室内側表面熱伝達層$\;(n_m=1)\;$は必ず確保。
    
    
- 各層に、以下の物性値等を設定する。

  - 厚さ$d_{m,n}\;[m]$
  - 熱伝導率$\lambda_{m,n} \; [W/(m･K)]$
  - 容積比熱$(c\rho)_{m,n} \; [kJ/(m^3･K)]$


- 表面熱伝達層や中空層の場合は、$(c\rho)_{m,n}=0\;[kJ/(m^3･K)]$とする。

- 中空層の場合は、その中空層の熱抵抗$r_{m,n} \; [m^2K/W]$で厚さ$d_{m,n}\;[m]$を除した値を、$\lambda_{m,n} \; [W/(m･K)]$として設定する($\lambda_{m,n} \leftarrow d_{m,n} / r_{m,n} $)。

- 表面熱伝達層の場合は、$d_{m,n}=1\;[m]$とし、表面熱伝達率$\alpha_s \; [W/(m^2･K)]$を$\lambda_{m,n}$として設定する($\lambda_{m,n} \leftarrow \alpha_s $)。


#### (2) 入力値

- 計算する単位応答の種類：$k$
  - $k=1$：$a$側温度励振$+b$側温度固定時の$a$側熱流応答($a$側励振時の応答)、$b$側温度励振$+a$側温度固定時の$a$側熱流応答($b$側励振時の応答)($a=1$、$b=N$)
  - $k=2$：$a$側表面熱流励振$+b$側温度固定時の$a$側表面温度応答($a$側励振時の応答)、$b$側温度励振$+a$側熱流固定時の$a$側表面温度応答($b$側励振時の応答)($a=2$、$b=N$)
  

- 一次元躯体構成$m$の表面積$\; [m^2]$：$S_m$   ($m=1$～$M$)
- 一次元躯体構成$m$の層数：$N_m$   ($m=1$～$M$)
- 一次元躯体構成$m$の層$1$～層$N$の厚さ$\;[m]$：$d_{m,n}$   ($m=1$～$M$, $n=1$～$N_m$)
- 一次元躯体構成$m$の層$1$～層$N$の熱伝導率$\; [W/(m･K)]$：$\lambda_{m,n}$   ($m=1$～$M$, $n=1$～$N_m$)
- 一次元躯体構成$m$の層$1$～層$N$の容積比熱$\; [kJ/(m^3･K)]$：$(c\rho)_{m,n}$   ($m=1$～$M$, $n=1$～$N_m$)
- 二組の周期のリスト$\; [hour]$：$\{ T_{1,i} \}$, $\{ T_{2,j} \}$     ($i=1$～, $j=1$～)
  - 周期リストの入力値として、`T_1` $ = \{ 0 \} \;$、`T_2` $ = \{ 0 \} \;$を指定すると、デフォルトの設定が適用される。
    - 適用されるデフォルト設定：`T_1` $ = \{ 1,2,3,4,6 \} $, `T_2` $ = \{ 24 \}$
    - 熱容量が大きい構成がある場合(暫定で$MAX(C_m) > 1000 [kJ/(m^2K)]\;$)：`T_1` $ = \{ 1,2,3,4,6,12,24 \} $, `T_2` $ = \{ 24,48,96 \} $


- 制御の初期値として設定するリスト：$\{ 0.0,\; 0.0 \}$

#### (3) 引数と入力値

`calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)`において、
  
- `kk` $ = k$
  
- 以下の$2$つの引数は、躯体構成数分($M$)分のリスト
  - `SSS` $ = [ S_1, \; S_2, \; \cdots, \; S_M ]$
  - `NNN` $ = [ N_1, \; N_2, \; \cdots, \; N_M ]$
    
    
- 以下の$3$つの引数は、層数分($N_m$)のリストを躯体構成数分($M$)結合したリスト
  - `d_ns` $ = [ d_{1,1}, \; d_{1,2}, \; \cdots, \; d_{1,N_1}, \; d_{2,1}, \; \cdots, \; d_{2,N_2},\; \cdots, \; d_{M,1}, \; \cdots, \; d_{M,N_M} \; ]$
  - `lambda_ns` $ = [ \lambda_{1,1}, \; \lambda_{1,2}, \; \cdots, \; \lambda_{1,N_1}, \; \lambda_{2,1}, \; \cdots, \; \lambda_{2,N_2},\; \cdots, \; \lambda_{M,1}, \; \cdots, \; \lambda_{M,N_M} \; ]$
  - `c_rho_ns` $ = [ (c\rho)_{1,1}, \; (c\rho)_{1,2}, \; \cdots, \; (c\rho)_{1,N_1}, \; (c\rho)_{2,1}, \; \cdots, \; (c\rho)_{2,N_2},\; \cdots, \; (c\rho)_{M,1}, \; \cdots, \; (c\rho)_{M,N_M} \; ]$ 
    
    
- 以下の$2$つの引数は、計算する周期を列挙したリスト
  - `T_1` $ = [ T_{1,1}, \; T_{1,2}, \; \cdots, \; ]$
  - `T_2` $ = [ T_{2,1}, \; T_{2,2}, \; \cdots, \; ]$  
    
    
- 以下の引数は、制御に使用するリストのため初期値を常に設定
  - `N_initBeta_1` $ = [0.0,\; 0.0]$  
    

#### (4) 出力値

- 一次元躯体の表面積合計値$\; [m^2]$：$S_{total}$
- 熱貫流率平均値$\; [W/(m^2K)]$：$U_{mean}$
- 一次元躯体の表面積$1m^2$あたりの熱容量$\; [kJ/(m^2K)]$：$C_{mean}$ 
- 表面$a$側の周期$24$時間における有効熱容量$\; [kJ/(m^2K)]$：$\kappa_{1, T=24hour}$  
- 算出した単位応答の種類：$k$
  - $k=1$：$a$側温度励振$+b$側温度固定時の$a$側熱流応答($a$側励振時の応答)、$b$側温度励振$+a$側温度固定時の$a$側熱流応答($b$側励振時の応答)($a=1$、$b=N$)
  - $k=2$：$a$側表面熱流励振$+b$側温度固定時の$a$側表面温度応答($a$側励振時の応答)、$b$側温度励振$+a$側熱流固定時の$a$側表面温度応答($b$側励振時の応答)($a=2$、$b=N$)
  
  
- $a$側励振の単位応答算定時に採用された二組の周期$\; [hour]$：$T_{a,1}$, $T_{a,2}$  
- $a$側励振単位応答の定常項$\; ([W/(m ^ 2K)].or.[m ^ 2K/W].or.[-])$：$B_{a,0}$
- $a$側励振単位応答の指数項$2$項の係数$\; ([W/(m ^ 2K)].or.[m ^ 2K/W].or.[-])$：$B_{a,1}$, $B_{a,2}$
- $a$側励振単位応答の指数項$2$項の時定数$[1/sec]$：$\beta_{a,1}$、$\beta_{a,2}$
  - ↑ 指数項二項からなる単位応答：$h = B_{a,0} + B_{a,1} \; e ^ {- \beta_{a,1} \; t} + B_{a,2} \; e ^ {- \beta_{a,2} \; t}$ における
  
  
- $b$側励振の単位応答算定時に採用された二組の周期$\; [hour]$：$T_{b,1}$, $T_{b,2}$  
- $b$側励振単位応答の定常項$\; ([W/(m ^ 2K)].or.[m ^ 2K/W].or.[-])$：$B_{b,0}$
- $b$側励振単位応答の指数項$2$項の係数$\; ([W/(m ^ 2K)].or.[m ^ 2K/W].or.[-])$：$B_{b,1}$, $B_{b,2}$
- $b$側励振単位応答の指数項$2$項の時定数$[1/sec]$：$\beta_{b,1}$、$\beta_{b,2}$
  - ↑ 指数項二項からなる単位応答：$h = B_{b,0} + B_{b,1} \; e ^ {- \beta_{b,1} \; t} + B_{b,2} \; e ^ {- \beta_{b,2} \; t}$ における
  
  

#### (5) 戻り値と出力値

`Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)` において、
  
- `Stotal` $ = S_{total}$
- `UUm` $ = U_{mean}$    
- `CCm` $ = C_{mean}$  
- `kappa24` $ = \kappa_{1, T=24hour}$  
- `kk` $ = k$ 
- `T1a` $ = T_{a,1}$ 
- `T2a` $ = T_{a,2}$ 
- `B0a` $ = B_{a,0}$ 
- `B1a` $ = B_{a,1}$ 
- `Beta1a` $ = \beta_{a,1}$ 
- `B2a` $ = B_{a,2}$ 
- `Beta2a` $ = \beta_{a,2}$ 
- `T1b` $ = T_{b,1}$ 
- `T2b` $ = T_{b,2}$  
- `B0b` $ = B_{b,0}$ 
- `B1b` $ = B_{b,1}$ 
- `Beta1b` $ = \beta_{b,1}$ 
- `B2b` $ = B_{b,2}$ 
- `Beta2b` $ = \beta_{b,2}$ 

### 1.2 計算の流れ

#### (1) フロー図

<img src="CalcFlow.png">

#### (2) 熱貫流率、熱容量、単位応答の定常項の計算式

2.2 (6)をあわせて参照のこと

- 躯体$1$～$M$の表面積合計値$S_{total} \; [m^2]$：

$$ S_{total} = \sum_{m=1}^{M} S_m \qquad (1)$$

- 躯体$1$～$M$の熱貫流率平均値$U_{mean} \; [W/(m^2･K)]$：

$$ U_{mean} = \frac{1}{S_{total}} \sum_{m=1}^{M} \bigl( S_m U_m \bigr) = \frac{1}{S_{total}} \sum_{m=1}^{M} \biggl( S_m \bigg/ \sum_{n=1}^{N_m} \frac{ d_{m,n} }{ \lambda_{m,n} }  \biggr) \qquad (2)$$

- 躯体$1$～$M$の表面積$1m^2$あたりの熱容量平均値$C_{mean} \; [kJ/(m^2･K)]$：

$$ C_{mean} = \frac{1}{S_{total}} \sum_{m=1}^{M} \bigl( S_m C_m \bigr) = \frac{1}{S_{total}} \sum_{m=1}^{M} \biggl( S_m \sum_{n=1}^{N_m} (c\rho)_{m,n} \: d_{m,n} \biggr) \qquad (3)$$

- 【$k=1$】躯体$m$の単位応答の定常項$\; B_{m,a,0}$, $B_{m,b,0}\;[W/(m ^ 2K)]$：

$$ B_{m,a,0}=B_{m,b,0}=U_m=1 \bigg/ \sum_{n=1}^{N_m} \frac{ d_{m,n} }{ \lambda_{m,n} } \qquad (4)$$

- 【$k=2$】躯体$m$の単位応答の定常項$\; B_{m,a,0}\;[m ^ 2K/W]$, $B_{m,b,0}\;[-]$：

$$ B_{m,a,0} = \sum_{n=2}^{N_m} \frac{ d_{m,n} }{ \lambda_{m,n} } \qquad (5)$$
$$ B_{m,b,0} = 1 \qquad (6)$$

- 【$k=1$】躯体$1$～$M$の単位応答定常項の平均値$\; B_{a,0}$, $B_{b,0}\; [W/(m ^ 2K)]$：

$$ B_{a,0}=B_{b,0}=U_{mean} \qquad (7)$$

- 【$k=2$】躯体$1$～$M$の単位応答定常項の平均値$\; B_{a,0}\;[m ^ 2K/W]$, $B_{b,0}\;[-]$：

$$ B_{a,0} =  \frac{1}{S_{total}} \sum_{m=1}^{M} \bigl( S_m B_{m,a,0}  \bigr) \qquad (8)$$
$$ B_{b,0} = 1 \qquad (9)$$

#### (3) 周波数応答を計算する周期の設定

- 二組の周期のリスト$\; [hour]$：$\{ T_{1,i} \}$, $\{ T_{2,j} \}$     ($i=1$～, $j=1$～)
- 周波数応答の計算は、$T \;( \in \{ T_{1,i} \} \cup \{ T_{2,j} \} \cup \{ 24 \} )\;$ について行う。
  - $T$の中に、$ 24[hour] $ を明示して入れているのは、有効熱容量算定を$24$時間周期時で行うため。
    
    
- 単位応答の計算は、$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の組合せで行う。
  - ただし、$ T_{1,i} = T_{2,j} $ の場合は、単位応答の計算はパスされる。


- 周期リストの入力値として、`T_1` $ = \{ 0 \} \;$、`T_2` $ = \{ 0 \} \;$を設定した場合には、それぞれデフォルトの設定が適用される。
  - 適用されるデフォルト設定：`T_1` $ = \{ 1,2,3,4,6 \} $, `T_2` $ = \{ 24 \}$
  - 熱容量が大きい構成がある場合(暫定で$MAX(C_m) > 1000 [kJ/(m^2K)]\;$)：`T_1` $ = \{ 1,2,3,4,6,12,24 \} $, `T_2` $ = \{ 24,48,96 \} $
  

- 単位応答が適切に求まる周期の組合せが少ないときには、計算する周期を追加して再計算する((9)参照)。
  - $\{ 0.5 \times T_{1,i} \} \cup \{ 2 \times T_{1,i} \}, \{ 0.5 \times T_{2,j} \} \cup \{ 2 \times T_{2,j} \} \;$ を、 $\{ T_{1,i} \}$, $\{ T_{2,j} \}\;$ に追加する。
  
  
- 角周波数$\omega \: [rad/s] \:$は、周期$T\: [hour]\: $から下式で求める。

$$\omega = 2 \pi / (T \times 3600) \qquad (10)$$



#### (4) 躯体$m$の周期毎の周波数応答(a側励振時 & b側励振時)の算出


2.2 (1)～(4)をあわせて参照のこと

- 周期$T \;( \in \{ T_{1,i} \} \cup \{ T_{2,j} \} \cup \{ 24 \} )\;$ に対応した$\omega$(式$(10)$)について、周波数応答を算出する。
  - ISO 13786-2007 "Thermal performance of building components - Dynamic thermal　characteristics - Calculation methods" の計算方法がベース
  

- 本稿内では、下記について、入力時と計算時で単位が異なる。中途で変換されるので注意すること。
   - 時間$T$：入力時は$[hour]$立て、計算時は基本的に角周波数$[rad/s]$を用いているので表出しないが$[s]$立てとなる。
   - 容積比熱$c\rho$：入力時は$[kJ/(m^3･K)]$、計算時は$[J/(m^3･K)]$。

##### a) 躯体単層の行列の計算式

$(c\rho)_{m,n} \neq 0$の単層については、角周波数$\omega$、各層の厚さ$d_{m,n}$、熱伝導率$\lambda_{m,n}$、容積比熱$(c\rho)_{m,n}$から$\textbf{Z}_{m,n}$を算出する。

- 各層の侵入厚さ$\:\delta[m]\:$：

$$ \delta_{m,n} = \sqrt {\frac{\lambda_{m,n} T}{\pi (c\rho)_{m,n}}} = \sqrt {\frac{2 \lambda_{m,n}}{\omega (c\rho)_{m,n}}} \qquad (11) \; (\leftarrow 2.2 式(11'))$$

- 各層の侵入厚さの比$\:\xi\:$：

$$ \xi_{m,n} = \frac{d_{m,n}}{\delta_{m,n}} = d_{m,n} \sqrt {\frac{\pi (c\rho)_{m,n}}{\lambda_{m,n} T}} = d_{m,n} \sqrt {\frac{\omega (c\rho)_{m,n}}{2 \lambda_{m,n}}} \qquad (12) \; (\leftarrow 2.2 式(a))$$

- 躯体単層($\:(c\rho)_{m,n} > 0\:$)の四端子行列$\;\textbf{Z}_{m,n} = \begin{pmatrix} Z_{11,m,n} & Z_{12,m,n} \\ Z_{21,m,n} & Z_{22,m,n} \end{pmatrix}\;$ の要素：


$$ \begin{align}
Z_{11,m,n} = Z_{22,m,n} &= \cosh(\xi_{m,n}) \cos(\xi_{m,n}) + j \sinh(\xi_{m,n}) \sin(\xi_{m,n}) \\ 
Z_{12,m,n} &= - \frac{\delta_{m,n}}{2 \lambda_{m,n}} \{ \sinh(\xi_{m,n}) \cos(\xi_{m,n}) + \cosh(\xi_{m,n}) \sin(\xi_{m,n}) \\
& \hspace{30pt} + j \;[ \cosh(\xi_{m,n}) \sin(\xi_{m,n}) - \sinh(\xi_{m,n}) \cos(\xi_{m,n}) ] \} \\
Z_{21,m,n} &= - \frac{\lambda_{m,n}}{\delta_{m,n}} \{\sinh(\xi_{m,n}) \cos(\xi_{m,n}) - \cosh(\xi_{m,n}) \sin(\xi_{m,n})\\ 
& \hspace{30pt} + j \; [\sinh(\xi_{m,n}) \cos(\xi_{m,n}) + \cosh(\xi_{m,n}) \sin(\xi_{m,n})]\} \qquad (13) \; (\leftarrow 2.2 式(14))  \\
\end{align} $$

##### b) 表面熱伝達層、中空層の行列の計算式

$(c\rho)_{m,n} = 0$の表面熱伝達層、中空層については、熱抵抗$R_{m,n} = d_{m,n}\:/\:\lambda_{m,n}$を用いて、下式で$\textbf{Z}_{m,n}$を算出する。

- 表面熱伝達層、中空層($\:(c\rho)_{m,n} = 0\:$)の四端子行列：
  
$$ \;\textbf{Z}_{m,n} = \begin{pmatrix} 1 & -R_{m,n} \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} 1 & -d_{m,n}/\lambda_{m,n} \\ 0 & 1 \end{pmatrix}  \qquad (14) \; (\leftarrow 2.2 式(15'), \; (18')) $$

##### c) 躯体全体の四端子行列の計算式

「室内側表面熱伝達層を含む」単位応答を求める際は$n_m=1$～$N_m$の、「室内側表面熱伝達層を含まない」単位応答を求める際は$n_m=2$～$N_m$の各層の行列をかけあわせて算出する。

- 層$1$～$N_m$の四端子行列$\;\textbf{Z}_m\;$：

  - 「室内側表面熱伝達層を含む」単位応答を求める際に使用($a=1$、$b=N_m$)

$$\textbf{Z}_m  = \begin{pmatrix} Z_{11,m} & Z_{12,m} \\ Z_{21,m} & Z_{22,m} \end{pmatrix} =\textbf{Z}_{m,N} \; \textbf{Z}_{m,N-1} \;  \cdots \;  \textbf{Z}_{m,2} \;  \textbf{Z}_{m,1}  \qquad (15) \; (\leftarrow 2.2 式(16'')) $$

- 層$2$～$N_m$の四端子行列$\;\textbf{Z}_{m2N}\;$：

  - 「室内側表面熱伝達層を含まない」単位応答を求める際に使用($a=2$、$b=N_m$)

$$\textbf{Z}_{m2N} = \begin{pmatrix} {Z_{11,m}}' & {Z_{12,m}}' \\ {Z_{21,m}}' & {Z_{22,m}}' \end{pmatrix} = \textbf{Z}_{m,N} \; \textbf{Z}_{m,N-1} \;  \cdots \;  \textbf{Z}_{m,2} \qquad (16) \; (\leftarrow 2.2 式(16''')) $$


##### d) 周波数応答の係数の計算式

導出過程については2.5 (6)を参照のこと。

$k=1$の場合は、$a$側温度励振における$a$側熱流応答$\;\hat{q}_{m,a}\;$、$b$側温度励振における$a$側熱流応答$\;\hat{q}_{m,b}\;$の係数を、$\;\textbf{Z}_m\;$の構成要素から求める。

$k=2$の場合は、$a$側熱流励振$+b$側温度固定における$a$側表面温度応答$\;\hat{\theta}_{m,a}\;$、$b$側温度励振$+a$側熱流固定における$a$側表面温度応答$\;\hat{\theta}_{m,ar}\;$を、$\;\textbf{Z}_{m2N}\;$の構成要素から求める。

- 【$k=1$】
- $a$側で単位温度振幅で励振する場合($\hat{\theta}_a = \cos \omega t$、$\hat{\theta}_b = 0$)の熱流応答：式$(17）$
- 式$(17)$を周波数応答$\; H_* = A_* \cos \varphi_* \cos \omega t - A_* \sin \varphi_* \sin \omega t \;$に当てはめた際の係数：式$(18)$


$$
\left\{
\begin{array}{ll}
\hat{q}_{m,a} = ( \: |Z_{11,m}| \: \big/ \: |Z_{12,m}|  \;) \; \cos \{ \omega t + \arg(Z_{11,m}) - \arg(Z_{12,m}) + \pi \}
\\
\hat{q}_{m,b} =  ( \: 1 \: \big/ \: |Z_{12,m}|  \;) \; \cos \{ \omega t - \arg(Z_{12,m}) - \pi \}
\end{array}
\right.  \qquad (17) \; (\leftarrow 2.2 式(g))
$$

$$
\left\{
\begin{array}{ll}
A_{m,a} \cos \varphi_{m,a} =( \: |Z_{11,m}| \: \big/ \: |Z_{12,m}|  \;) \;\cos \{ \arg(Z_{11,m}) - \arg(Z_{12,m}) + \pi \}
\\
A_{m,a} \sin \varphi_{m,a} =( \: |Z_{11,m}| \: \big/ \: |Z_{12,m}|  \;) \;\sin \{ \arg(Z_{11,m}) - \arg(Z_{12,m}) + \pi \}
\\
A_{m,b} \cos \varphi_{m,b} = ( \: 1 \: \big/ \: |Z_{12,m}|  \;) \; \cos \{ - \arg(Z_{12,m}) - \pi \} \\
A_{m,b} \sin \varphi_{m,b} = ( \: 1 \: \big/ \: |Z_{12,m}|  \;) \; \sin \{ - \arg(Z_{12,m}) - \pi \} \\
\end{array}
\right.  \qquad (18) \; (\leftarrow 2.2 式(g'))
$$

- ここで、$|Z_{**}|$は、$Z_{**}$の大きさ$\bigl( \:|Z_{**}| = \sqrt{\Re(Z_{**})^2 + \Im(Z_{**})^2} \: \bigr)$である。
- また、$\arg(Z_{**})$は、$Z_{**}$の偏角$\bigl( \: \arg(Z_{**}) = \tan^{-1} \{\Im(Z_{**}) \: \big/ \: \Re(Z_{**})\} \: \bigr)$である。

- 【$k=2$】
- $a$側が熱流の単位振幅励振、$b$側が温度固定とする場合($\hat{q}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$)の$a$側温度応答$\;\hat{\theta}_{m,a}\;$：式$(19)$第$1$式
- $b$側が温度の単位振幅励振、$a$側が熱流固定とする場合($\hat{\theta}_b = \cos \omega t$、$\hat{q}_a = 0\:$)の$a$側温度応答$\;\hat{\theta}_{m,ar}\;$：式$(19)$第$2$式
- 式$(19)$を周波数応答$\; H_* = A_* \cos \varphi_* \cos \omega t - A_* \sin \varphi_* \sin \omega t \;$に当てはめた際の係数：式$(20)$
  - $\hat{\theta}_{m,ar}$は貫流応答の相反則により、$a$側が熱流の単位振幅励振、$b$側が温度固定とする場合($\hat{q}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$)の$b$側熱流応答$\hat{q}_{m,b}$と同式となる(2.5節の式$(j)$第2式と式$(m)$第1式)。

$$
\left\{
\begin{array}{ll}
\hat{\theta}_{m,a} = ( \: |{Z_{12,m}}'| \: \big/ \: |{Z_{11,m}}'|  \;) \; \cos \{ \omega t + \arg({Z_{12,m}}') - \arg({Z_{11,m}}') + \pi \}
\\
\hat{q}_{m,b} = \hat{\theta}_{m,ar} = ( \: 1 \: \big/ \: |{Z_{11,m}}'|  \;) \; \cos \{ \omega t - \arg({Z_{11,m}}') \}
\end{array}
\right.  \qquad (19) \; (\leftarrow 2.2 式(j))
$$

$$
\left\{
\begin{array}{ll}
A_{m,a} \cos \varphi_{m,a} = ( \: |{Z_{12,m}}'| \: \big/ \: |{Z_{11,m}}'|  \;) \; \cos \{ \arg({Z_{12,m}}') - \arg({Z_{11},m}') + \pi \} \\
    A_{m,a} \sin \varphi_{m,a} = ( \: |{Z_{12,m}}'| \: \big/ \: |{Z_{11,m}}'|  \;) \; \sin \{ \arg({Z_{12,m}}') - \arg({Z_{11,m}}') + \pi \} \\
A_{m,b} \cos \varphi_{m,b} = ( \: 1 \: \big/ \: |{Z_{11,m}}'|  \;) \; \cos \{ - \arg({Z_{11,m}}') \} \\
A_{m,b} \sin \varphi_{m,b} = ( \: 1 \: \big/ \: |{Z_{11,m}}'|  \;) \; \sin \{ - \arg({Z_{11,m}}') \} \\
\end{array}
\right.  \qquad (20) \; (\leftarrow 2.2 式(j'))
$$

#### (5) 面積重み付け平均の周波数応答の計算式

- 躯体$m$の周波数応答の係数$A_{m,*} \cos \varphi_{m,*}$,$ A_{m,*} \sin \varphi_{m,*}$を面積で重み付け平均して躯体$1$～$M$を統合した周波数応答の係数を算定する。

$$ A_* \cos \varphi_* = \frac{1}{S_{total}} \sum_{m=1}^{M} \bigl( S_m A_{m,*} \cos \varphi_{m,*} \bigr)  \qquad (21)$$
$$ A_* \sin \varphi_* = \frac{1}{S_{total}} \sum_{m=1}^{M} \bigl( S_m A_{m,*} \sin \varphi_{m,*} \bigr)  \qquad (22)$$

#### (6) 有効熱容量の計算式

2.2 (5)をあわせて参照のこと。導出過程については2.5 (7)を参照のこと。

- 室内側($n=1$側)の単位面積あたりの有効熱容量$\kappa_1 \:[J/(m^2･K)]$：

  - 有効熱容量は、$k$に関わらず、「表面熱伝達層を含む」構成で求めることとする。
    - 式$(18)$の係数を、式$(21)$、式$(22)$で面積重み付け平均した係数から算定する。

$$ \kappa_1 = \frac{1}{\omega} \sqrt { (A_a \sin \varphi_a  - A_b \sin \varphi_b) ^ 2 + (A_a \cos \varphi_a - A_b \cos \varphi_b ) ^ 2 } \qquad (23) \; (\leftarrow 2.2 式(n)) \\ $$ 

- 有効熱容量は、$24$時間周期における周波数応答から算出する。




#### (7) 単位応答の係数の算出

3.2をあわせて参照のこと。導出過程については3.5 (1)(2)を参照のこと。

- 単位応答の計算は、二組の周期のリスト$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の組合せから抽出される($T_1$, $T_2$)で行う。
  - ただし、$ T_1 = T_2 $ の場合は、単位応答の計算はパスされる。
  - 角周波数$\omega_1$, $\omega_2$は、式$(10)$で算出する。
  

- 式$(26)$で$\beta_1$、$\beta_2$を収束計算から求める ($\beta_1 < \beta_2$、$\beta_m > 0$)。
- 本稿内での式$(26)$の収束計算の手順は以下の通り。
   - $\beta_1$の初期値は$10 ^ {-10}$。収束の状況によって異なる値を設定できる。
   - 式$(26)$で、$\beta_1$から$\beta_2$を、$\beta_2$から$\beta_1$を計算して1ステップ。
   - $\beta_1$、$\beta_2$の双方について、前ステップの値との差が$10 ^ {-10}$を下回れば収束計算終了。
   - 計算収束後に$\beta_1 < \beta_2$となるように大小関係を整える。
   - 計算収束後に$\beta_1 = \beta_2$となる場合は、($T_1$, $T_2$)の周波数応答からの計算は失敗となる。
   - 計算収束後に$\beta_m \leq 0$となる場合も計算失敗となる。
   
   
- 二組の周期のリスト$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の組合せの中で、単位応答の時定数が適切に求まる周期の組合せが少ないときには、式$(26)$の収束計算の初期値を再設定して二組の周期のリストすべてで再計算し(ただし、$ T_1 = T_2 $除く)、適当な$\beta_1$、$\beta_2$の求まる確度を上げている((9)参照)。
  - 再計算で設定する$\beta_1$初期値は、次項(8)で最も誤差が少ないと評価された単位応答の$\beta_1$とする。
  - 式$(26)$の収束計算で一組も適切に収束しなかった場合は、(9)記載の周期を拡張しての再計算に移行する。
  - $\beta_1$初期値を変更した再計算でも単位応答が適切に求まる周期の組合せが少ないと評価されたときにも、(9)記載の周期を拡張しての再計算に移行する。
  - 単位応答が適切に求まる周期の組合せが少ないと評価するのは、式$(26)$の収束計算が適切に終わる回数が、全計算回数の$2$割に届かないこととしている(暫定)。
  

- $\beta_1$、$\beta_2$が定まった後、式$(27)$で$B_1$、$B_2$を求める。
- 西澤、三浦：周波数特性から単位応答を近似的に求める方法に関する検討, 日本建築学会大会学術講演梗概集D2, pp.586-587, 2017

$$ \qquad
C_p = A_p \cos \varphi_p - B_0 = \sum_{m=1}^M \frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2} \quad ( p = 1,\! 2 ) \quad (24) \; (\leftarrow 3.2 式(4))\\
S_p = A_p \sin \varphi_p = \sum_{m=1}^M \frac{B_m \beta_m \omega_p}{{\beta_m}^2 + {\omega_p}^2}  \quad ( p = 1,\! 2 ) \quad (25) \; (\leftarrow 3.2 式(5))\\
$$
$$ \qquad 
\beta_m
= \frac{\omega_p {\omega_q}^2 S_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 \omega_q S_q ( {\beta_n}^2 + {\omega_q}^2 )}
{{\omega_q}^2 C_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 C_q ( {\beta_n}^2 + {\omega_q}^2 )}
, \quad
( m = 1,\! 2,\; n = 3 - m, \; p = 1, \; q = 2) \\
\hspace{250pt}  (26) \; (\leftarrow 3.2 式(8))
\\
$$
$$ \qquad 
B_m
= \frac{{\beta_m}^2 + {\omega_p}^2}{{\omega_p}^2 ( \beta_n - \beta_m )} ( \beta_n C_p - \omega_p S_p )
, \quad
( m = 1,\! 2,\; n = 3 - m, \; p = 1,\! 2) \\
\hspace{250pt}  (27) \; (\leftarrow 3.2 式(9))
\\
$$


#### (8) 最小誤差となる単位応答の抽出

二組の周期リスト$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の組合せで前項(7)で求めた単位応答の、(4)で求めた周期$T \;( \in \{ T_{1,i} \} \cup \{ T_{2,j} \} \cup \{ 24 \} )\;$ の周波数応答(精算値)に対する誤差を算出し、最も誤差が小さくなる単位応答を抽出する。

- 前項(7)で$(T_1, T_2)$の組合せ(ただし、$ T_1 = T_2 $除く)で求めた単位応答から、周期$T$、角周波数$\omega$における周波数応答の係数$A' \cos \varphi'$、$A' \sin \varphi'$を下式から算出する。

$$ 
A' \cos \varphi' =  B_0 +  \sum_{m=1}^2 \frac{B_m {\omega}^2}{{\beta_m}^2 + {\omega}^2} \quad (24') \\
A' \sin \varphi' = \sum_{m=1}^2 \frac{B_m \beta_m \omega}{{\beta_m}^2 + {\omega}^2}  \quad (25') \\
$$


- (4)で求めた周期$T$ の周波数応答(精算値)との平均平方二乗誤差$RMSE$を、その係数$A \cos \varphi$、$A \sin \varphi$を用いて下式で計算する(導出過程は3.5 (3)を参照のこと)。

$$ RMSE = \sqrt{ \frac{ (A' \cos \varphi' - A \cos \varphi)^2 + (A' \sin \varphi' - A \sin \varphi)^2 }{2} } \qquad (28) \; (\leftarrow 3.5 式(k))$$


- この$RMSE$を、${T} = ( \in \{ T_{1,i} \} \cup \{ T_{2,j} \} \cup \{ 24 \} )\;$ で計算し、最大となる誤差を$RMSE_{MAX}$とする。

- 二組の周期リスト$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の組合せ(ただし、$ T_1 = T_2 $除く)の各々において計算した$RMSE_{MAX}$から、最小となる$RMSE_{MAX}$をもつ$(T_1, T_2)$の周波数応答から得られた単位応答を、周波数応答との誤差が最小となる単位応答として抽出し、採用することとする。


- 二組の周期のリスト$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の組合せの中で、単位応答が適切に求まる周期の組合せが少ないときに行う式$(26)$の再計算には、本項で最も誤差が少ないと評価された単位応答の$\beta_1$を初期値として設定する。


#### (9) 単位応答抽出の確度を上げるための再計算

単位応答の確度を上げるために、以下の再計算ループを取り入れている。

- 式$(26)$の収束計算の結果、適切に単位応答の時定数$\beta_1$、$\beta_2$が求まるケースが少ない場合に、$\beta_1$の初期値を再設定して式$(26)$の収束を再計算するループ
- 周期$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の取り方が不十分で適切な単位応答が求まるケースが少ない場合に、周期$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ を拡張して再計算するループ

a) $\beta_1$の収束の再計算ループ

- (7)において、二組の周期のリスト$\{ T_{1,i} \}$ と $\{ T_{2,j} \}$ の組合せの中で、単位応答の時定数が適切に求まる周期の組合せが少ないときに実施する再計算ループ
- 単位応答が適切に求まる周期の組合せが少ないと評価するのは、式$(26)$の収束計算が適切に終わる回数が、全計算回数の$2$割に届かないこととしている(暫定)。
- 再計算で設定する$\beta_1$初期値は、(8)で最も誤差が少ないと評価された単位応答の$\beta_1$とする。 
- 式$(26)$の収束計算で一組も適切に収束しなかった場合は、b)記載の周期を拡張しての再計算に移行する。
- $\beta_1$初期値を変更した再計算でも単位応答が適切に求まる周期の組合せが少ないと評価されたときにも、b)記載の周期を拡張しての再計算に移行する。

b) 周期拡張の再計算ループ

- a)のループを1回行っても変わらずに単位応答が適切に求まる周期の組合せが少ないと評価されたときには、計算対象としている周期二組のリストが不十分であると考え、(3)でのリスト拡張の対応をとり、再計算する。
- $\{ 0.5 \times T_{1,i} \} \cup \{ 2 \times T_{1,i} \}, \{ 0.5 \times T_{2,j} \} \cup \{ 2 \times T_{2,j} \} \;$ を、 $\{ T_{1,i} \}$, $\{ T_{2,j} \}\;$ に追加する。
- このループを4回追加して行っても適切に求まらない場合は、再計算を打ち切り、その時点で最も適切と判断された単位応答を持って結果とする。

### 1.3 計算プログラム

In [20]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算するプログラム本体 """
import cmath
import numpy as np

def calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1):
    
    """ 各躯体構成での熱貫流率、熱容量、単位応答の定常項作成 (1.2 (2))"""
    NStart = 0
    Stotal = 0
    CCmax = 0
    UU = [0 for i1 in range(len(SSS))] 
    UUm = 0
    CC = [0 for i1 in range(len(SSS))] 
    CCm = 0
    B0 = [[[0 for i3 in range(2)] for i2 in range(2)] for i1 in range(len(SSS))] 
    B0m = [[0 for i3 in range(2)] for i2 in range(2)]  
    for m in range(len(SSS)):   #SSSの要素数:一組の単位応答にまとめる躯体構成の数
                                #m:躯体構成の番号
        SS = SSS[m]                               #SS:躯体mの表面積[m2]
        Stotal += SS
        NN = NNN[m]                               #NN:躯体mの層数N
        d_n = d_ns[NStart:NStart+NN]              #d_nに抽出→躯体mの各層の厚さd[m]
        lambda_n = lambda_ns[NStart:NStart+NN]    #lambda_nに抽出→躯体mの各層の熱伝導率λ[W/(mK)]
        c_rho_n =  c_rho_ns[NStart:NStart+NN]     #c_rho_nに抽出→躯体mの容積比熱cρ[kJ/(m3K)] 
        #2.3 (4)のモジュール呼び出し
        UU[m], CC[m], B0[m][0][0], B0[m][0][1], B0[m][1][0], B0[m][1][1] = calc_u_c_b0(d_n, lambda_n, c_rho_n)
        UUm += UU[m] * SS
        CCm += CC[m] * SS
        for k in range(2):
            for j in range(2):
                B0m[k][j] += B0[m][k][j] * SS
        CCmax = CC[m] if CCmax < CC[m] else CCmax
        NStart += NN                              #次の躯体m+1のためにNの開始位置を更新  
    
    UUm /= Stotal
    CCm /= Stotal
    for k in range(2):
        for j in range(2):
            B0m[k][j] /= Stotal
    
    """ 2対の周期の組T1,T2[hour]の設定 (1.2 (3)) """ 
    # 引数で0を設定した場合→2対の組を自動的設定 """ 
    if T1[0] == 0:
        T1 = [1,2,3,4,6] if CCmax < 1000 else [1,2,3,4,6,12,24]   
    if T2[0] == 0:
        T2 = [24] if CCmax < 1000 else [24,48,96]   #熱容量が大きいときには24時間より長い周期も追加する。
    TT = []
    for T in (T1 + T2 + [24]):    #有効熱容量算出に24時間を使うので欠けることがないように追加
        if T not in TT:
            TT.append(T)
    TT.sort()     #ダブりを排除した周期Tのリストを作成
    omega = [0 for i in range(len(TT))]
    for i in range(len(TT)):
        omega[i] = 2 * cmath.pi / (TT[i] * 3600)
    
    """ 各躯体構成で周波数応答を算定 (1.2 (4)) """
    NStart = 0
    Stotal = 0
    AC = [[[[0 for i4 in range(2)] for i3 in range(2)] for i2 in range(len(TT))] for i1 in range(len(SSS))]
    AS = [[[[0 for i4 in range(2)] for i3 in range(2)] for i2 in range(len(TT))] for i1 in range(len(SSS))]
    ACm = [[[0 for i4 in range(2)] for i3 in range(2)] for i2 in range(len(TT))] 
    ASm = [[[0 for i4 in range(2)] for i3 in range(2)] for i2 in range(len(TT))] 
    kappa = [[0 for i2 in range(len(TT))] for i1 in range(len(SSS))]
    kappa_10 = [0 for i2 in range(len(TT))] 
        
    for m in range(len(SSS)):                 #SSSの要素数:一組の単位応答にまとめる躯体構成の数
        SS = SSS[m]                               #SS:躯体mの表面積[m2]
        Stotal += SS
        NN = NNN[m]                               #NN:躯体mの層数N
        d_n = d_ns[NStart:NStart+NN]              #d_nに抽出→躯体mの各層の厚さd[m]
        lambda_n = lambda_ns[NStart:NStart+NN]    #lambda_nに抽出→躯体mの各層の熱伝導率λ[W/(mK)]
        c_rho_n =  c_rho_ns[NStart:NStart+NN]     #c_rho_nに抽出→躯体mの容積比熱cρ[kJ/(m3K)]
        NStart += NN                              #次の躯体m+1のためにNの開始位置を更新  
        
        """ 周期毎に周波数応答を算定 """ 
        # 2章本体プログラムを叩かず、直接四端子行列モジュールを直接叩くことに変更
        for i in range(len(TT)):        
            ZZ, ZZ_2_N = calc_z(d_n, lambda_n, c_rho_n, omega[i])
            
            # AC[躯体m][周期Ti][k-1][0]：躯体m、周期Tiにおける「k」のa側励振の応答の係数Acosφ|a
            # AS[躯体m][周期Ti][k-1][0]：躯体m、周期Tiにおける「k」のa側励振の応答の係数Asinφ|a
            # AC[躯体m][周期Ti][k-1][1]：躯体m、周期Tiにおける「k」のb側励振の応答の係数Acosφ|b
            # AS[躯体m][周期Ti][k-1][1]：躯体m、周期Tiにおける「k」のb側励振の応答の係数Asinφ|b
            # kappa[躯体m][周期Ti]：躯体m、周期Tiにおけるa側温度励振に対するa側有効熱容量κa
            
            # k=1:   # n=1表面温度励振、n=N温度固定→n=1吸熱応答、n=N貫流応答
            rr_a = abs(ZZ[0,0]) / abs(ZZ[0,1])              # k=1のa側応答の振幅(2.2 式(g)第1式)
            pp_a = cmath.phase(ZZ[0,0]) - cmath.phase(ZZ[0,1]) + cmath.pi       
                                                            # k=1のa側応答の位相差(2.2 式(g)第1式)
            AC[m][i][0][0] = rr_a * np.cos(pp_a)            # k=1のa側応答のAcosφ(2.2 式(g')第1式)
            AS[m][i][0][0] = rr_a * np.sin(pp_a)            # k=1のa側応答のAsinφ(2.2 式(g')第2式)
            rr_b = 1 / abs(ZZ[0,1])                         # k=1のb側応答の振幅(2.2 式(g)第2式)
            pp_b = - cmath.phase(ZZ[0,1]) - cmath.pi        # k=1のb側応答の位相差(2.2 式(g)第2式)
            AC[m][i][0][1] = rr_b * np.cos(pp_b)            # k=1のb側応答のAcosφ(2.2 式(g')第3式)
            AS[m][i][0][1] = rr_b * np.sin(pp_b)            # k=1のb側応答のAsinφ(2.2 式(g')第4式)
            
            # k=2:   # n=2表面熱流励振、n=N温度固定→n=2表面温度応答、n=N熱流応答
            rr_a_2 = abs(ZZ_2_N[0,1]) / abs(ZZ_2_N[0,0])    # k=2のa側応答の振幅(2.2 式(j)第1式)
            pp_a_2 = cmath.phase(ZZ_2_N[0,1]) - cmath.phase(ZZ_2_N[0,0]) + cmath.pi  
                                                            # k=2のa側応答の位相差(2.2 式(j)第1式)
            AC[m][i][1][0] = rr_a_2 * np.cos(pp_a_2)        # k=2のa側応答のAcosφ(2.2 式(g')第1式)
            AS[m][i][1][0] = rr_a_2 * np.sin(pp_a_2)        # k=2のa側応答のAsinφ(2.2 式(g')第2式)
            rr_b_2 = 1 / abs(ZZ_2_N[0,0])                   # k=2のb側応答の振幅(2.2 式(j)第2式)
            pp_b_2 = - cmath.phase(ZZ_2_N[0,0])             # k=2のb側応答の位相差(2.2 式(j)第2式)
            AC[m][i][1][1] = rr_b_2 * np.cos(pp_b_2)        # k=2のb側応答のAcosφ(2.2 式(g')第1式)
            AS[m][i][1][1] = rr_b_2 * np.sin(pp_b_2)        # k=2のb側応答のAsinφ(2.2 式(g')第2式)

            # 2.2 式(22')の計算：室内側有効熱容量κ1[kJ/(m3k)]の算定
            kappa[m][i] = 1 / omega[i] * abs( ( ZZ[0,0] - 1 ) / ZZ[0,1] ) / 1000  
            
            """ 複数躯体の周波数応答を表面積をかけて加算 """ 
            for k in range(2):
                for j in range(2):
                    ACm[i][k][j] += AC[m][i][k][j] * SS
                    ASm[i][k][j] += AS[m][i][k][j] * SS
    
    """ 複数躯体の周波数応答を面積加重平均 (1.2 (5)) """ 
    for i in range(len(TT)):
        for k in range(2):
            for j in range(2):
                ACm[i][k][j] /= Stotal
                ASm[i][k][j] /= Stotal
    
    """ 複数躯体で合成した周波数応答(n=1表面含む)から有効熱容量を算定 (1.2 (6)) """ 
    for i in range(len(TT)):
        k = 1-1
        # 室内側有効熱容量κ1[kJ/(m3k)]
        kappa_10[i] = 1 / omega[i] * ( ( ASm[i][k][0] - ASm[i][k][1] )**2 
                                     + ( ACm[i][k][0] - ACm[i][k][1] )**2 )**0.5 / 1000

    kappa24 = kappa_10[TT.index(24)]
    
    
    BB = [[[[0 for i4 in range(2)] for i3 in range(2)] for i2 in range(len(T2))] for i1 in range(len(T1))]
    Beta = [[[[0 for i4 in range(2)] for i3 in range(2)] for i2 in range(len(T2))] for i1 in range(len(T1))]
    calcN = [[[0 for i3 in range(2)] for i2 in range(len(T2))] for i1 in range(len(T1))]
    initerrormax = 9999
    RMSE = [[[[initerrormax for i4 in range(len(TT))] for i3 in range(2)] for i2 in range(len(T2))] \
                                    for i1 in range(len(T1))]
    RMSEmax = [[[initerrormax for i3 in range(2)] for i2 in range(len(T2))] for i1 in range(len(T1))]
    RMSEmax0 = [initerrormax for i1 in range(2)]
    ii10 = [0 for i1 in range(2)]
    ii20 = [0 for i1 in range(2)]
    calcR = [0 for i1 in range(2)]
        
    """ 角周波数二組に対応した周波数応答を走査抽出し、指数項二項で近似した単位応答を算出 (1.2 (7)) """    
    k = kk - 1
    calcRthreshold = 0.2
    Nrecalc = 2
    for j in range(2):
        calcR[j] = 0
        
        #  1.2 (9) a)の計算ループ
        while ((calcR[j] < calcRthreshold) \
                and ((N_initBeta_1[j] - ((N_initBeta_1[j] // 1000) * 1000)) // 100 < Nrecalc)):
            
            # print (calcR[j], (N_initBeta_1[j] - ((N_initBeta_1[j] // 1000) * 1000)) // 100)
            if N_initBeta_1[j] - N_initBeta_1[j]//100 == 0:
                initBeta_1 = 10**(-10)      # Betaの収束計算の初期値に指定がない場合の設定
            else:
                initBeta_1 = N_initBeta_1[j] - (N_initBeta_1[j] // 100) * 100
            calcNtotal = 0.0
            calcNN = 0.0
            for T_1, T_2 in [(T_1,T_2) for T_1 in T1 for T_2 in T2]:
                if not T_1==T2:
                    i1 = TT.index(T_1)
                    i2 = TT.index(T_2)
                    ii1 = T1.index(T_1)
                    ii2 = T2.index(T_2)
                    calcNtotal += 1
                    # 2組の周波数応答から指数項2項の単位応答を計算
                    BB[ii1][ii2][j][0], BB[ii1][ii2][j][1], Beta[ii1][ii2][j][0], Beta[ii1][ii2][j][1] \
                            ,calcN[ii1][ii2][j], Diff_Beta_1, Diff_Beta_2 \
                        = calc_unitresponse( initBeta_1, B0m[k][j], omega[i1], ACm[i1][k][j], ASm[i1][k][j] \
                            ,omega[i2], ACm[i2][k][j], ASm[i2][k][j] )
                    # print(BB[ii1][ii2][j][0], BB[ii1][ii2][j][1], Beta[ii1][ii2][j][0], Beta[ii1][ii2][j][1],calcN[ii1][ii2][j])
                
                    # 算出した単位応答から周波数応答を復号した際に、周波数応答精算値との誤差を計算 (1.2 (8))
                    if calcN[ii1][ii2][j] > 0:
                        calcNN += 1
                        for i in range(len(TT)):
                            AC_1 = calc_ac_p0(omega[i], B0m[k][j], [ BB[ii1][ii2][j][0], BB[ii1][ii2][j][1] ]
                                     , [ Beta[ii1][ii2][j][0], Beta[ii1][ii2][j][1] ] )
                            AS_1 = calc_as_p0(omega[i], [ BB[ii1][ii2][j][0], BB[ii1][ii2][j][1] ]
                                     , [ Beta[ii1][ii2][j][0], Beta[ii1][ii2][j][1] ] )
                            RMSE[ii1][ii2][j][i]=calc_rmse(AC_1, AS_1, ACm[i][k][j], ASm[i][k][j])
                            # print(TT[i],RMSE[ii1][ii2][j][i])
                            if RMSEmax[ii1][ii2][j] == initerrormax:
                                RMSEmax[ii1][ii2][j] = RMSE[ii1][ii2][j][i]
                            elif RMSEmax[ii1][ii2][j] < RMSE[ii1][ii2][j][i]:
                                RMSEmax[ii1][ii2][j] = RMSE[ii1][ii2][j][i]
                            
            """ 指数項二項で近似した単位応答が作る周波数応答の誤差→採用する単位応答を決定する (1.2 (8)) """
            calcR[j] = calcNN / calcNtotal
            # print(N_initBeta_1[j],RMSEmax[ii1][ii2][j],calcNN, calcNtotal,calcR[j])   
            if calcNN == 0:
                N_initBeta_1[j] = ((N_initBeta_1[j]  // 1000) + 1 ) * 1000.0 + (Nrecalc + 1) *100.0
            else:
                for T_1, T_2 in [(T_1,T_2) for T_1 in T1 for T_2 in T2]:
                    if not T_1==T_2:
                        ii1 = T1.index(T_1)
                        ii2 = T2.index(T_2)
                        if RMSEmax0[j] >= RMSEmax[ii1][ii2][j]:
                            RMSEmax0[j] = RMSEmax[ii1][ii2][j]
                            ii10[j] = ii1
                            ii20[j] = ii2
                N_initBeta_1[j] = ((N_initBeta_1[j]  // 100) + 1 ) * 100.0 + Beta[ii10[j]][ii20[j]][j][0]
                # print (j,calcR[j],calcNN,calcNtotal,N_initBeta_1[j])

    """ 出力 """
    j = 0    #a側励振に対する応答
    B0a = B0m[k][j]
    B1a = BB[ii10[j]][ii20[j]][j][0]
    Beta1a = Beta[ii10[j]][ii20[j]][j][0]
    B2a = BB[ii10[j]][ii20[j]][j][1]
    Beta2a = Beta[ii10[j]][ii20[j]][j][1]
    T1a = T1[ii10[j]]
    T2a = T2[ii20[j]]
    
    j = 1    #b側励振に対する応答
    B0b = B0m[k][j]
    B1b = BB[ii10[j]][ii20[j]][j][0]
    Beta1b = Beta[ii10[j]][ii20[j]][j][0]
    B2b = BB[ii10[j]][ii20[j]][j][1]
    Beta2b = Beta[ii10[j]][ii20[j]][j][1]
    T1b = T1[ii10[j]]
    T2b = T2[ii20[j]]
    
    """ 計算が不十分であると判断したときに、周期T1、T2を増やして再帰実行 (1.2 (9) a)) """
    N_initBeta_1[0] = (max([N_initBeta_1[0],N_initBeta_1[1]]) // 100) * 100.0 \
                + (N_initBeta_1[0] - (N_initBeta_1[0] // 100) * 100.0)
    N_initBeta_1[1] = (N_initBeta_1[0] // 100) * 100.0 \
                + (N_initBeta_1[1] - (N_initBeta_1[1] // 100) * 100.0)
    # print(N_initBeta_1[0],N_initBeta_1[1])

    if (N_initBeta_1[0] // 1000 > 4) or (min([calcR[0],calcR[1]]) > calcRthreshold): 
        pass
    elif N_initBeta_1[j] // 100 - ((N_initBeta_1[j] // 1000) * 10) >= Nrecalc:
        for j in range(2):
            N_initBeta_1[j] = ((N_initBeta_1[j] // 1000) + 1) *1000.0 \
                    + (N_initBeta_1[j] - (N_initBeta_1[j] // 100) * 100.0)
        for T in [T_1*0.5 for T_1 in T1] + [T_1*2 for T_1 in T1]:
            if T not in T1:
                T1.append(T)
        T1.sort()     #拡大した周期T1のリスト(ダブり除外)を作成
        for T in [T_2*0.5 for T_2 in T2] + [T_2*2 for T_2 in T2]:
            if T not in T2:
                T2.append(T)
        T2.sort()     #拡大した周期T2のリスト(ダブり除外)を作成
        # print(T1,T2)

        [Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
            = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)  

    return [Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b]  


### 1.4 計算例



In [21]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その1, 「高断熱・大熱容量床」のn=1表面熱伝達層を含む場合の熱流応答(1対の周期から)) """ 
kk = 1                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [6]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448,  0.012,   0.1,    0.045,  1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.260, 715.806, 13.395, 25.116, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [2]
T2 = [24]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=73.97939071[kJ/(m2K)]   セルR23C26
# a側応答
# B0 = 0.212934526　　　　セルR84C2
# B1 = 5.348575591　　　　セルR114C18
# B2 = 0.801010388　　　　セルR114C19
# Beta1 = 2.23046E-05　　 セルR114C16
# Beta2 = 0.000620977　　 セルR114C17
# b側応答
# B0 = 0.212934526　　　　セルR164C2
# B1 = -0.250692349　　　 セルR194C18
# B2 = 0.040236803　　　　セルR194C19
# Beta1 = 2.39631E-05　　 セルR194C16
# Beta2 = 0.000177816　　 セルR194C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:2[m2], 熱貫流率U=0.21293452605868463[W/(m2K)], 熱容量C=266.0684368[kJ/(m2K)], a側有効熱容量(24時間周期)κ=73.97939009726598[kJ/(m2K)]
k=1,T1=2,T2=24[h]の周波数応答から算出した、a側励振に対する単位応答[W/(m2K)]：
ha = 0.21293452605868463 + 5.348575625697582 * e^( -2.230464886553243e-05 t) + 0.8010103626961922 * e^( -0.0006209771126678378 t)
k=1,T1=2,T2=24[h]の周波数応答から算出した、b側励振に対する単位応答[W/(m2K)]：
hb = 0.21293452605868463 - 0.2506920928916584 * e^( -2.3963088904129675e-05 t) + 0.04023655429388204 * e^( -0.00017781679518954832 t)


In [22]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その2, 「高断熱・大熱容量床」のn=1表面熱伝達層を含まない場合, 表面温度応答(1対の周期から)) """ 
kk = 2                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [6]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448,  0.012,   0.1,    0.045,  1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.260, 715.806, 13.395, 25.116, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [2]
T2 = [24]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=73.97939071[kJ/(m2K)]   セルR23C26
# a側応答
# B0 = 4.547025536　　　　セルR244C2
# B1 = -4.518220113　　　 セルR274C18
# B2 = -0.020888737　　　 セルR274C19
# Beta1 = 8.49748E-07　　 セルR274C16
# Beta2 = 0.000545504　　 セルR274C17
# b側応答
# B0 = 1　　　　          セルR324C2
# B1 = -1.005740046　　　 セルR354C18
# B2 = 0.006166047　　　　セルR354C19
# Beta1 = 9.06877E-07　　 セルR354C16
# Beta2 = 0.000175593　　 セルR354C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:2[m2], 熱貫流率U=0.21293452605868463[W/(m2K)], 熱容量C=266.0684368[kJ/(m2K)], a側有効熱容量(24時間周期)κ=73.97939009726598[kJ/(m2K)]
k=2,T1=2,T2=24[h]の周波数応答から算出した、a側励振に対する単位応答[m2K/W]：
ha = 4.547025535854562 - 4.518220114425538 * e^( -8.497477544326384e-07 t) - 0.020888736255855907 * e^( -0.0005455038950999095 t)
k=2,T1=2,T2=24[h]の周波数応答から算出した、b側励振に対する単位応答[-]：
hb = 1.0 - 1.005740021803063 * e^( -9.068766721096147e-07 t) + 0.006166023380935349 * e^( -0.0001755943717902467 t)


In [23]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その3, 「高断熱・大熱容量床」のn=1表面熱伝達層を含む場合の熱流応答(複数周期への対応確認)) """ 
kk = 1                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [6]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448,  0.012,   0.1,    0.045,  1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.260, 715.806, 13.395, 25.116, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [0.5,1,2,3,4,6,8,12]
T2 = [18,24,36,48]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=73.97939071[kJ/(m2K)]   セルR23C26
# a側応答
# T1 =1, T2 =18 が採用    R104行
# B0 = 0.212934526　　　　セルR84C2
# B1 = 5.368346793　　　  セルR104C18
# B2 = 0.82932731　　　   セルR104C19
# Beta1 = 2.28314E-05　　 セルR104C16
# Beta2 = 0.000795025　　 セルR104C17

# b側応答
# T1 =3, T2 =48 が採用    R204行
# B0 = 0.212934526　　　　セルR164C2
# B1 = -0.244185068　　　 セルR204C18
# B2 = 0.035588947　　　　セルR204C19
# Beta1 = 2.23426E-05　　 セルR204C16
# Beta2 = 0.000254197　　 セルR204C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:2[m2], 熱貫流率U=0.21293452605868463[W/(m2K)], 熱容量C=266.0684368[kJ/(m2K)], a側有効熱容量(24時間周期)κ=73.97939009726598[kJ/(m2K)]
k=1,T1=1,T2=18[h]の周波数応答から算出した、a側励振に対する単位応答[W/(m2K)]：
ha = 0.21293452605868463 + 5.368346825674695 * e^( -2.2831431681480115e-05 t) + 0.8293272812706411 * e^( -0.0007950249495930699 t)
k=1,T1=3,T2=48[h]の周波数応答から算出した、b側励振に対する単位応答[W/(m2K)]：
hb = 0.21293452605868463 - 0.2441849531290322 * e^( -2.2342575210825894e-05 t) + 0.03558882771813247 * e^( -0.00025419826015785215 t)


In [24]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その4, 「高断熱・大熱容量床」のn=1表面熱伝達層を含まない場合, 表面温度応答(複数周期への対応確認)) """ 
kk = 2                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [6]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448,  0.012,   0.1,    0.045,  1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.260, 715.806, 13.395, 25.116, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [0.5,1,2,3,4,6,8,12,18,24]
T2 = [18,24,36,48,60,72,96,120,144]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=73.97939071[kJ/(m2K)]   セルR23C26
# a側応答
# T1 =1, T2 =18 が採用    R264行
# B0 = 4.547025536　　　　セルR244C2
# B1 = -4.518552449　　　 セルR264C18
# B2 = -0.021735389　　　 セルR264C19
# Beta1 = 8.67489E-07　　 セルR26416
# Beta2 = 0.000695019　　 セルR264C17

# b側応答
# T1 =3, T2 =48 が採用    R364行
# B0 = 1　　　　          セルR324C2
# B1 = -1.005161732　　　 セルR364C18
# B2 = 0.005927093　　　　セルR364C19
# Beta1 = 8.50578E-07　　 セルR364C16
# Beta2 = 0.000241826　　 セルR364C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))


総面積:2[m2], 熱貫流率U=0.21293452605868463[W/(m2K)], 熱容量C=266.0684368[kJ/(m2K)], a側有効熱容量(24時間周期)κ=73.97939009726598[kJ/(m2K)]
k=2,T1=1,T2=36[h]の周波数応答から算出した、a側励振に対する単位応答[m2K/W]：
ha = 4.547025535854562 - 4.51821038923942 * e^( -8.493633737838383e-07 t) - 0.022061090139916195 * e^( -0.0006868777597746268 t)
k=2,T1=3,T2=144[h]の周波数応答から算出した、b側励振に対する単位応答[-]：
hb = 1.0 - 1.0051079739679774 * e^( -8.439916877669204e-07 t) + 0.005873146638580396 * e^( -0.00024309211604605038 t)


C:\NishiHPSx360_Pub\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [25]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その5, 「高断熱・小熱容量床」のn=1表面熱伝達層を含む場合の熱流応答) """ 
kk = 1                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [5]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.012,   0.1,    0.045,  1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     715.806, 13.395, 25.116, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
T2 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱小熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=9.295070697[kJ/(m2K)]   セルR23C26
# a側応答
# T1 =0.5, T2 =2 が採用    R88行
# B0 = 0.216814894　　　　セルR84C2
# B1 = 5.203363838　　　 セルR88C18
# B2 = 0.506453057　　　 セルR288C19
# Beta1 = 0.000625766　　 セルR88C16
# Beta2 = 0.004003056　　 セルR88C17

# b側応答
# T1=2[h], T2=6[h] が採用    R190行
# B0 = 0.216814894        セルR164C2
# B1 = -3.285426682　　　 セルR190C18
# B2 = 3.092807354　　　　セルR190C19
# Beta1 = 0.000665884　　 セルR190C16
# Beta2 = 0.000742165　　 セルR190C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

C:\NishiHPSx360_Pub\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


総面積:2[m2], 熱貫流率U=0.21681489407128907[W/(m2K)], 熱容量C=11.059392[kJ/(m2K)], a側有効熱容量(24時間周期)κ=9.29505791186453[kJ/(m2K)]
k=1,T1=2,T2=0.5[h]の周波数応答から算出した、a側励振に対する単位応答[W/(m2K)]：
ha = 0.21681489407128907 + 5.203368154526522 * e^( -0.0006257665271795937 t) + 0.506450251404618 * e^( -0.004003093323911136 t)
k=1,T1=6,T2=2[h]の周波数応答から算出した、b側励振に対する単位応答[W/(m2K)]：
hb = 0.21681489407128907 - 3.2815187670581074 * e^( -0.0006658457117661881 t) + 3.0888998622892956 * e^( -0.0007422222109542629 t)


In [26]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その6, 「高断熱・小熱容量床」のn=1表面熱伝達層を含まない場合, 表面温度応答) """ 
kk = 2                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [5]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.012,   0.1,    0.045,  1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     715.806, 13.395, 25.116, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
T2 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱小熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=9.295070697[kJ/(m2K)]   セルR23C26
# a側応答
# T1=1[h], T2=8[h]が採用    R262行
# B0 = 4.462975536　　　　セルR244C2
# B1 = -4.429648342　　　 セルR262C18
# B2 = -0.010810198　　　 セルR262C19
# Beta1 = 2.42403E-05　　 セルR262C16
# Beta2 = 0.00160349　　 セルR262C17

# b側応答
# T1=1[h], T2=18[h] が採用    R190行
# B0 = 1       セルR324C2
# B1 = -1.04847293　　　 セルR344C18
# B2 = 0.052948088  　　　セルR344C19
# Beta1 = 2.43041E-05　　 セルR344C16
# Beta2 = 0.000740116　　 セルR344C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:2[m2], 熱貫流率U=0.21681489407128907[W/(m2K)], 熱容量C=11.059392[kJ/(m2K)], a側有効熱容量(24時間周期)κ=9.29505791186453[kJ/(m2K)]
k=2,T1=1,T2=8[h]の周波数応答から算出した、a側励振に対する単位応答[m2K/W]：
ha = 4.462975535854562 - 4.429648506164922 * e^( -2.424027826490577e-05 t) - 0.010810070600229394 * e^( -0.0016035103786051087 t)
k=2,T1=18,T2=1[h]の周波数応答から算出した、b側励振に対する単位応答[-]：
hb = 1.0 - 1.0484725857243393 * e^( -2.4304077319221893e-05 t) + 0.05294773834544129 * e^( -0.0007401230457220353 t)


C:\NishiHPSx360_Pub\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [27]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その7, 「低断熱・大熱容量床」のn=1表面熱伝達層を含む場合の熱流応答) """ 
kk = 1                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [5]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448, 0.012,   0.02,   1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,     0.16,    0.038,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.26, 715.806, 56.511, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
T2 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(低断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=73.84839656[kJ/(m2K)]   セルR23C26
# a側応答
# T1=1[h], T2=18[h] が採用    R104行
# B0 = 1.016391083　　　　セルR84C2
# B1 = 4.587813235　　　 セルR104C18
# B2 = 0.808906803　　　 セルR104C19
# Beta1 = 2.77768E-05　　 セルR104C16
# Beta2 = 0.000814868　　 セルR104C17

# b側応答
# T1=2[h], T2=18[h] が採用    R193行
# B0 = 1.016391083        セルR164C2
# B1 = -1.140129265　　　 セルR193C18
# B2 = 0.13565027　　　　セルR193C19
# Beta1 = 2.75048E-05　　 セルR193C16
# Beta2 = 0.000352894　　 セルR193C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:2[m2], 熱貫流率U=1.0163910826972273[W/(m2K)], 熱容量C=264.7289368[kJ/(m2K)], a側有効熱容量(24時間周期)κ=73.84839656084777[kJ/(m2K)]
k=1,T1=18,T2=1[h]の周波数応答から算出した、a側励振に対する単位応答[W/(m2K)]：
ha = 1.0163910826972273 + 4.587813234517744 * e^( -2.7776801207264965e-05 t) + 0.8089068061523673 * e^( -0.0008148677855349111 t)
k=1,T1=18,T2=2[h]の周波数応答から算出した、b側励振に対する単位応答[W/(m2K)]：
hb = 1.0163910826972273 - 1.1401292615538696 * e^( -2.7504799726768135e-05 t) + 0.13565026229103486 * e^( -0.0003528941564011122 t)


C:\NishiHPSx360_Pub\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [28]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その8, 「低断熱・大熱容量床」のn=1表面熱伝達層を含まない場合, 表面温度応答) """ 
kk = 2                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [5]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448, 0.012,   0.02,   1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,     0.16,    0.038,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.26, 715.806, 56.511, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
T2 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(低断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=73.84839656[kJ/(m2K)]   セルR23C26
# a側応答
# T1=1[h], T2=18[h] が採用    R264行
# B0 = 0.834619521 　　　セルR244C2
# B1 = -0.806855088　　　 セルR264C18
# B2 = -0.021087606　　　 セルR264C19
# Beta1 = 5.01731E-06　　 セルR264C16
# Beta2 = 0.000714974　　 セルR264C17

# b側応答
# T1=2[h], T2=24[h] が採用    R354行
# B0 = 1                  セルR324C2
# B1 = -1.021333679　　　 セルR354C18
# B2 = 0.023375906　　　　セルR354C19
# Beta1 = 4.94837E-06　　 セルR354C16
# Beta2 = 0.000328532　　 セルR354C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:2[m2], 熱貫流率U=1.0163910826972273[W/(m2K)], 熱容量C=264.7289368[kJ/(m2K)], a側有効熱容量(24時間周期)κ=73.84839656084777[kJ/(m2K)]
k=2,T1=18,T2=1[h]の周波数応答から算出した、a側励振に対する単位応答[m2K/W]：
ha = 0.8346195208169677 - 0.8068550875627502 * e^( -5.017310771644009e-06 t) - 0.0210876059389123 * e^( -0.0007149743481811532 t)
k=2,T1=24,T2=2[h]の周波数応答から算出した、b側励振に対する単位応答[-]：
hb = 1.0 - 1.021333679395233 * e^( -4.948373993885737e-06 t) + 0.02337590619898704 * e^( -0.0003285319126502798 t)


C:\NishiHPSx360_Pub\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [29]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その9, 「低断熱・小熱容量床」のn=1表面熱伝達層を含む場合の熱流応答) """ 
kk = 1                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [4]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.012,   0.02,   1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   0.16,    0.038,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     715.806, 56.511, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
T2 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(低断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=7.29526529[kJ/(m2K)]   セルR23C26
# a側応答
# T1=0.5[h], T2=2[h] が採用    R104行
# B0 = 1.111329361　　　　セルR84C2
# B1 = 4.530083855　　　 セルR88C18
# B2 = 0.675627949　　　 セルR88C19
# Beta1 = 0.000805727　　 セルR88C16
# Beta2 = 0.014682172　　 セルR88C17

# b側応答
# T1=0.5[h], T2=2[h] が採用    R168行
# B0 = 1.11132936        セルR164C2
# B1 = -1.366566071　　　 セルR168C18
# B2 = 0.30134692　　　　セルR168C19
# Beta1 = 0.000806418　　 セルR168C16
# Beta2 = 0.007693396　　 セルR168C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

C:\NishiHPSx360_Pub\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


総面積:2[m2], 熱貫流率U=1.1113293611820425[W/(m2K)], 熱容量C=9.719892[kJ/(m2K)], a側有効熱容量(24時間周期)κ=7.295265289841202[kJ/(m2K)]
k=1,T1=0.5,T2=2[h]の周波数応答から算出した、a側励振に対する単位応答[W/(m2K)]：
ha = 1.1113293611820425 + 4.53008385498319 * e^( -0.0008057268387522219 t) + 0.6756279486738226 * e^( -0.014682172437505402 t)
k=1,T1=0.5,T2=2[h]の周波数応答から算出した、b側励振に対する単位応答[W/(m2K)]：
hb = 1.1113293611820425 - 1.366566071187711 * e^( -0.0008064178137202401 t) + 0.30134691907574584 * e^( -0.00769339574957115 t)


In [30]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その10, 「低断熱・大熱容量床」のn=1表面熱伝達層を含まない場合, 表面温度応答) """ 
kk = 2                                                          #単位応答の種別
SSS = [2]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [4]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.012,   0.02,   1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   0.16,    0.038,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     715.806, 56.511, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
T2 = [48,36,24,18,12,8,6,4,3,2,1,0.5]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(低断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=7.29526529[kJ/(m2K)]   セルR23C26
# a側応答
# T1=0.5[h], T2=3[h] が採用    R249行
# B0 = 0.750569521　　　　セルR244C2
# B1 = -0.724399578　　　 セルR249C18
# B2 = -0.017117327　　　 セルR249C19
# Beta1 = 0.000157874　　 セルR249C16
# Beta2 = 0.013173881　　 セルR249C17

# b側応答
# T1=0.5[h], T2=3[h] が採用    R329行
# B0 = 1                  セルR324C2
# B2 = -1.041624842　　　　セルR329C19
# B1 = 0.050271504　　　 セルR329C18
# Beta2 = 0.000157913　　 セルR329C17     #Excel上では、B1とB2、Beta1とBeta2が逆になっている
# Beta1 = 0.007493086　　 セルR329C16

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
          .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

C:\NishiHPSx360_Pub\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


総面積:2[m2], 熱貫流率U=1.1113293611820425[W/(m2K)], 熱容量C=9.719892[kJ/(m2K)], a側有効熱容量(24時間周期)κ=7.295265289841202[kJ/(m2K)]
k=2,T1=3,T2=0.5[h]の周波数応答から算出した、a側励振に対する単位応答[m2K/W]：
ha = 0.7505695208169677 - 0.7243995783203915 * e^( -0.00015787386910459344 t) - 0.017117326956129376 * e^( -0.013173881044514295 t)
k=2,T1=0.5,T2=3[h]の周波数応答から算出した、b側励振に対する単位応答[-]：
hb = 1.0 - 1.0416248423939745 * e^( -0.00015791295236335594 t) + 0.050271504049938444 * e^( -0.007493085785705922 t)


In [31]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その11, 「3種類の床」のn=1表面熱伝達層を含む場合の熱流応答) """ 
kk = 1                                                          #単位応答の種別
SSS = [2,4,5]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [6,4,5]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1,     0.13448,  0.012,   0.1,    0.045,  1]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0,     1896.260, 715.806, 13.395, 25.116, 0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
d_ns +=     [1,     0.13,     0.1,     1]       #2組目の各層の厚さd[m]
lambda_ns +=[6.7,   1.8,      0.04,    6.7]     #2組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns += [0,     2000,     12,      0]       #2組目の各層の容積比熱cρ[kJ/(m3K)] 
d_ns +=     [1,     0.13,     0.01,    0.1,     1]       #3組目の各層の厚さd[m]
lambda_ns +=[6.7,   1.8,      0.1,     0.04,    6.7]     #3組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns += [0,     2000,     300,     12,      0]       #3組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [0.5,1,2,3,4,6,8,12]
T2 = [18,24,36,48]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床3種) .xlsm 内の
#                「【3組統合】周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=75.36163045[kJ/(m2K)]   セルR23C26
# a側応答
# T1=1[h], T2=18[h] が採用    R104行
# B0 = 0.318393745　　　　セルR84C2
# B1 = 5.392270344　　　 セルR104C18
# B2 = 0.726964877　　　 セルR104C19
# Beta1 = 2.37554E-05　　 セルR104C16
# Beta2 = 0.00085501　　 セルR104C17

# b側応答
# T1=2[h], T2=18[h] が採用    R193行
# B0 = 0.318393745       セルR164C2
# B1 = -0.350732095　　　 セルR193C18
# B2 = 0.03568142　　　　セルR193C19
# Beta1 = 2.37988E-05　　 セルR193C16
# Beta2 = 0.000364093　　 セルR193C17

[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
         .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:11[m2], 熱貫流率U=0.3183937445109369[W/(m2K)], 熱容量C=263.44880669090907[kJ/(m2K)], a側有効熱容量(24時間周期)κ=75.36163033622576[kJ/(m2K)]
k=1,T1=1,T2=18[h]の周波数応答から算出した、a側励振に対する単位応答[W/(m2K)]：
ha = 0.3183937445109369 + 5.3922703457225465 * e^( -2.3755412734619785e-05 t) + 0.7269648757245618 * e^( -0.0008550099447473438 t)
k=1,T1=2,T2=18[h]の周波数応答から算出した、b側励振に対する単位応答[W/(m2K)]：
hb = 0.3183937445109369 - 0.35073207586379596 * e^( -2.3798755122693847e-05 t) + 0.0356814056247362 * e^( -0.00036409280172966 t)


In [32]:
""" 一次元壁体構成から周波数応答を経て指数項二項で近似した単位応答を計算 """
""" 入力値(その12, 「3種類の床」のn=1表面熱伝達層を含まない場合, 表面温度応答) """ 
kk = 2                                                          #単位応答の種別
SSS = [2,4,5]                                                 #複数躯体構成｢1組｣のそれぞれの表面積[m2]
NNN = [6,4,5]                                                 #複数躯体構成｢1組｣のそれぞれの層数N
d_ns =      [1.0,   0.13448,  0.012,   0.1,    0.045,  1.0]       #1組目の各層の厚さd[m]
lambda_ns = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]     #1組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns =  [0.0,   1896.260, 715.806, 13.395, 25.116, 0.0]       #1組目の各層の容積比熱cρ[kJ/(m3K)] 
d_ns +=     [1.0,   0.13,     0.1,     1.0]     #2組目の各層の厚さd[m]
lambda_ns +=[6.7,   1.8,      0.04,    6.7]     #2組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns += [0.0,   2000.0,   12.0,    0.0]     #2組目の各層の容積比熱cρ[kJ/(m3K)] 
d_ns +=     [1.0,   0.13,     0.01,    0.1,     1.0]     #3組目の各層の厚さd[m]
lambda_ns +=[6.7,   1.8,      0.1,     0.04,    6.7]     #3組目の各層の熱伝導率λ[W/(mK)]
c_rho_ns += [0.0,   2000.0,   300.0,   12.0,    0.0]     #3組目の各層の容積比熱cρ[kJ/(m3K)] 
T1 = [0.5,1,2,3,4,6,8,12]
T2 = [18,24,36,48]
N_initBeta_1 = [0.0 ,0.0]     # 制御に使用。初期値として[0.0 ,0.0]を入れておくこと。

""" Excelでの出力値 """ 
# セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床3種) .xlsm 内の
#                「【3組統合】周期定常→指数項二項近似」シート内
# a側有効熱容量(24時間周期)κ=75.36163045[kJ/(m2K)]   セルR23C26
# a側応答
# T1=1[h], T2=18[h] が採用    R264行
# B0 = 3.098848605　　　　セルR244C2
# B1 = -3.074106169　　　 セルR264C18
# B2 = -0.018653927　　　 セルR264C19
# Beta1 = 1.27376E-06　　 セルR264C16
# Beta2 = 0.000759439　　 セルR264C17

# b側応答
# T1=2[h], T2=24[h] が採用    R354行
# B0 = 1                  セルR324C2
# B1 = -1.005403068　　　 セルR354C18
# B2 = 0.005972567  　　　セルR354C19
# Beta1 = 1.31297E-06　　 セルR354C16
# Beta2 = 0.000344448　　 セルR354C17


[Stotal,UUm,CCm,kappa24,kk,T1a,T2a,B0a,B1a,Beta1a,B2a,Beta2a,T1b,T2b,B0b,B1b,Beta1b,B2b,Beta2b] \
    = calc_wall_to_unitresponse(kk, SSS, NNN, d_ns, lambda_ns, c_rho_ns, T1, T2, N_initBeta_1)

print('総面積:{}[m2], 熱貫流率U={}[W/(m2K)], 熱容量C={}[kJ/(m2K)], a側有効熱容量(24時間周期)κ={}[kJ/(m2K)]'
         .format(Stotal,UUm,CCm,kappa24) )
unit = '[W/(m2K)]' if kk==1 else '[m2K/W]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、a側励振に対する単位応答{}：'.format(kk,T1a,T2a,unit))
PM1 = '+' if B1a >= 0 else '-'
PM2 = '+' if B2a >= 0 else '-'
print('ha = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0a, PM1, abs(B1a), Beta1a, PM2, abs(B2a), Beta2a))
unit = '[W/(m2K)]' if kk==1 else '[-]'
print('k={},T1={},T2={}[h]の周波数応答から算出した、b側励振に対する単位応答{}：'.format(kk,T1b,T2b,unit))
PM1 = '+' if B1b >= 0 else '-'
PM2 = '+' if B2b >= 0 else '-'
print('hb = {} {} {} * e^( -{} t) {} {} * e^( -{} t)'
            .format(B0b, PM1, abs(B1b), Beta1b, PM2, abs(B2b), Beta2b))

総面積:11[m2], 熱貫流率U=0.3183937445109369[W/(m2K)], 熱容量C=263.44880669090907[kJ/(m2K)], a側有効熱容量(24時間周期)κ=75.36163033622576[kJ/(m2K)]
k=2,T1=1,T2=18[h]の周波数応答から算出した、a側励振に対する単位応答[m2K/W]：
ha = 3.098848604890789 - 3.074106169420801 * e^( -1.2737560247175707e-06 t) - 0.018653926397008105 * e^( -0.0007594392904731901 t)
k=2,T1=2,T2=24[h]の周波数応答から算出した、b側励振に対する単位応答[-]：
hb = 1.0 - 1.005403065649192 * e^( -1.3129704421620013e-06 t) + 0.005972564586302548 * e^( -0.000344447999959678 t)



## 2. 一次元躯体構成＋熱物性から周波数応答を算出するモジュール


ISO 13786-2007 "Thermal performance of building components - Dynamic thermal　characteristics - Calculation methods" の計算方法をベースに、一次元躯体構成から、ある角周波数に対応した周波数応答を算出する。


### 2.1 入力値と出力値

#### (1) 一次元躯体の設定方法

- 全体で$N$層：$N-2\;$層の躯体構成要素＋その両面の表面熱伝達層

  - 層$1 \; (n=1)$：単位応答を計算する側(例：外壁の室内側)の表面熱伝達層
  - 層$2$～層$N-1 \; (n=2$～$N-1)$：躯体を構成する部材の層(中空層含む)
  - 層$N \; (n=N)$：反対側(例：外壁の外気側)の表面熱伝達層
  - 「室内側表面熱伝達層を含まない」周波数応答・単位応答を求める際でも、室内側表面熱伝達層$\;(n=1)\;$は必ず確保。
    
    
- 各層に、以下の物性値等を設定する。

  - 厚さ$d_n\;[m]$
  - 熱伝導率$\lambda_n \; [W/(m･K)]$
  - 容積比熱$(c\rho)_n \; [kJ/(m^3･K)]$


- 表面熱伝達層や中空層の場合は、$(c\rho)_n=0\;[kJ/(m^3･K)]$とする。

- 中空層の場合は、その中空層の熱抵抗$r_n \; [m^2K/W]$で厚さ$d_n\;[m]$を除した値を、$\lambda_n \; [W/(m･K)]$として設定する($\lambda_n \leftarrow d_n / r_n $)。

- 表面熱伝達層の場合は、$d_n=1\;[m]$とし、表面熱伝達率$\alpha_s \; [W/(m^2･K)]$を$\lambda_n$として設定する($\lambda_n \leftarrow \alpha_s $)。


#### (2) 入力値

- 層$1$～層$N$の厚さ$\;[m]$：$d_n$
- 層$1$～層$N$の熱伝導率$\; [W/(m･K)]$：$\lambda_n$
- 層$1$～層$N$の容積比熱$\; [kJ/(m^3･K)]$：$(c\rho)_n$
- 角周波数$[rad/s]$：$\omega$
- 計算する単位応答の種類：$k$
  - $k=1$：$a$側温度励振$+b$側温度固定時の$a$側熱流応答、$b$側温度励振$+a$側温度固定時の$a$側熱流応答($a=1$、$b=N$)
  - $k=2$：$a$側表面熱流励振$+b$側温度固定時の$a$側温度応答、$b$側温度励振$+a$側熱流固定時の$a$側温度応答($a=2$、$b=N$)


#### (3) 引数と入力値

- `calc_frequencyresponse(kk, d_n, lambda_n, c_rho_n, omega)`において、
  
  - `kk` $ = k$
  - `d_n` $ = [ d_1, \; d_2, \; \cdots, \; d_N ]$
  - `lambda_n` $ = [ \lambda_1, \; \lambda_2, \; \cdots, \; \lambda_N ]$
  - `c_rho_n` $ = [ (c\rho)_1, \; (c\rho)_2, \; \cdots, \; (c\rho)_N ]$
  - `omega` $ = \omega$
    

#### (4) 出力値

- $a$側励振の$a$側周波数応答の係数：$A_a \cos \varphi_a$、$A_a \sin \varphi_a$ ← 周波数応答$\; H_a = A_a \cos \varphi_a \cos \omega t - A_a \sin \varphi_a \sin \omega t \;$における 

- $b$側励振の$a$側周波数応答の係数：$A_b \cos \varphi_b$、$A_b \sin \varphi_b$ ← 周波数応答$\; H_b = A_b \cos \varphi_b \cos \omega t - A_b \sin \varphi_b \sin \omega t \;$における
    
- $a$側励振の$a$側有効熱容量$\; [kJ/(m^3･K)]$：$\kappa_1$


#### (5) 戻り値と出力値

- `AC_a, AS_a, AC_b, AS_b, kappa_1 = calc_frequencyresponse(kk, d_n, lambda_n, c_rho_n, omega)`   において、
  
  - `AC_a` $ = A_a \cos \varphi_a$
  - `AS_a` $ = A_a \sin \varphi_a$    
  - `AC_b` $ = A_b \cos \varphi_b$  
  - `AS_b` $ = A_b \sin \varphi_b$  
  - `kappa_1` $ = \kappa_1$ 


### 2.2 計算の流れ

- 本稿内では、下記について、入力時と計算時で単位が異なる。中途で変換されるので注意すること。
   - 時間$T$：入力時は$[hour]$立て、計算時は基本的に角周波数$[rad/s]$を用いているので表出しないが$[s]$だてとなる。
   - 容積比熱$c\rho$：入力時は$[kJ/(m^3･K)]$、計算時は$[J/(m^3･K)]$。
   
   
- 式番号は2.5に準拠

#### (1) 躯体単層の行列の計算式

$(c\rho)_n \neq 0$の単層については、角周波数$\omega$、各層の厚さ$d_n$、熱伝導率$\lambda_n$、容積比熱$(c\rho)_n$から$\textbf{Z}_n$を算出する。

- 各層の侵入厚さ$\:\delta[m]\:$：

$$ \delta_n = \sqrt {\frac{\lambda_n T}{\pi (c\rho)_n}} = \sqrt {\frac{2 \lambda_n}{\omega (c\rho)_n}} \qquad (11')$$

- 各層の侵入厚さの比$\:\xi\:$：

$$ \xi_n = \frac{d_n}{\delta_n} = d_n \sqrt {\frac{\pi (c\rho)_n}{\lambda_n T}} = d_n \sqrt {\frac{\omega (c\rho)_n}{2 \lambda_n}} \qquad (a)$$

- 躯体単層($\:(c\rho)_n > 0\:$)の四端子行列$\;\textbf{Z}_n = \begin{pmatrix} Z_{11,n} & Z_{12,n} \\ Z_{21,n} & Z_{22,n} \end{pmatrix}\;$ の要素：

$$ \begin{align}
Z_{11,n} = Z_{22,n} &= \cosh(\xi_n) \cos(\xi_n) + j \sinh(\xi_n) \sin(\xi_n) \\ 
Z_{12,n} &= - \frac{\delta_n}{2 \lambda_n} \{ \sinh(\xi_n) \cos(\xi_n) + \cosh(\xi_n) \sin(\xi_n) + j [ \cosh(\xi_n) \sin(\xi_n) - \sinh(\xi_n) \cos(\xi_n) ] \} \\
Z_{21,n} &= - \frac{\lambda_n}{\delta_n} \{\sinh(\xi_n) \cos(\xi_n) - \cosh(\xi_n) \sin(\xi_n) + j [\sinh(\xi_n) \cos(\xi_n) + \cosh(\xi_n) \sin(\xi_n)]\}  \qquad (14) \\
\end{align} $$

#### (2) 表面熱伝達層、中空層の行列の計算式

$(c\rho)_n = 0$の表面熱伝達層、中空層については、熱抵抗$R_n = d_n/\lambda_n$を用いて、下式で$\textbf{Z}_n$を算出する。

- 表面熱伝達層、中空層($c \rho = 0$)の四端子行列：
  
$$ \;\textbf{Z}_n = \begin{pmatrix} 1 & -R_n \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} 1 & -d_n/\lambda_n \\ 0 & 1 \end{pmatrix}  \qquad (15'), \; (18') $$

#### (3) 表面熱伝達層、中空層の行列の計算式

「室内側表面熱伝達層を含む」単位応答を求める際は$n=1$～$N$の、「室内側表面熱伝達層を含まない」単位応答を求める際は$n=2$～$N$の各層の行列をかけあわせて算出する。

- 層$1$～$N$の四端子行列$\;\textbf{Z}\;$：

  - 「室内側表面熱伝達層を含む」単位応答を求める際に使用($a=1$、$b=N$)

$$\textbf{Z}  = \begin{pmatrix} Z_{11} & Z_{12} \\ Z_{21} & Z_{22} \end{pmatrix} = \begin{pmatrix} R_{11}+jI_{11} & R_{12}+jI_{12} \\ R_{21}+jI_{21} & R_{22}+jI_{22} \end{pmatrix}=\textbf{Z}_N \; \textbf{Z}_{N-1} \;  \cdots \;  \textbf{Z}_{2} \;  \textbf{Z}_{1}  \qquad (16'') $$

- 層$2$～$N$の四端子行列$\;\textbf{Z}_{2N}\;$：

  - 「室内側表面熱伝達層を含まない」単位応答を求める際に使用($a=2$、$b=N$)

$$\textbf{Z}_{2N} = \begin{pmatrix} {Z_{11}}' & {Z_{12}}' \\ {Z_{21}}' & {Z_{22}}' \end{pmatrix} = \begin{pmatrix} {R_{11}}'+j{I_{11}}' & {R_{12}}'+j{I_{12}}' \\ {R_{21}}'+j{I_{21}}' & {R_{22}}'+j{I_{22}}' \end{pmatrix}=\textbf{Z}_N \; \textbf{Z}_{N-1} \;  \cdots \;  \textbf{Z}_{2} \qquad (16''') $$

#### (4) 周波数応答の係数の計算式

導出過程は2.5 (6)を参照のこと。

$k=1$の場合は、$a$側温度励振における$a$側熱流応答$\;\hat{q}_a\;$、$b$側温度励振における$a$側熱流応答$\;\hat{q}_b\;$の係数を、$\;\textbf{Z}\;$の構成要素から求める。

$k=2$の場合は、$a$側熱流励振$+b$側温度固定における$a$側表面温度応答$\;\hat{\theta}_a\;$、$b$側温度励振$+a$側熱流固定における$a$側表面温度応答$\;\hat{\theta}_{ar}\;$を、$\;\textbf{Z}_{2N}\;$の構成要素から求める。

- 【$k=1$】
- $a$側で単位温度振幅で励振する場合($\hat{\theta}_a = \cos \omega t$、$\hat{\theta}_b = 0$)の熱流応答：式$(g）$
- 式$(g)$を周波数応答$\; H_* = A_* \cos \varphi_* \cos \omega t - A_* \sin \varphi_* \sin \omega t \;$に当てはめた際の係数：式$(g')$


$$
\left\{
\begin{array}{ll}
\hat{q}_a = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\cos \Bigl( \omega t +  \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi \Bigr)
\\
\hat{q}_b = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \; \cos \Bigl( \omega t - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi \Bigr)
\end{array}
\right.  \qquad (g)
$$

$$
\left\{
\begin{array}{ll}
A_a \cos \varphi_a = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\cos \Bigl( \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi \Bigr) \\
A_a \sin \varphi_a = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\sin \Bigl(   \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi \Bigr) \\
A_b \cos \varphi_b = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \; \cos \Bigl( - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi \Bigr) \\
A_b \sin \varphi_b = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \; \sin \Bigl( - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi \Bigr) \\
\end{array}
\right.  \qquad (g')
$$

- 【$k=2$】
- $a$側が熱流の単位振幅励振、$b$側が温度固定とする場合($\hat{q}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$)の$a$側温度応答$\;\hat{\theta}_a\;$：式$(j)$第$1$式
- $b$側が温度の単位振幅励振、$a$側が熱流固定とする場合($\hat{\theta}_b = \cos \omega t$、$\hat{q}_a = 0\:$)の$a$側温度応答$\;\hat{\theta}_{ar}\;$：式$(j)$第$2$式
- 式$(j)$を周波数応答$\; H_* = A_* \cos \varphi_* \cos \omega t - A_* \sin \varphi_* \sin \omega t \;$に当てはめた際の係数：式$(j')$
  - $\hat{\theta}_{ar}$は貫流応答の相反則により、$a$側が熱流の単位振幅励振、$b$側が温度固定とする場合($\hat{q}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$)の$b$側熱流応答$\hat{q}_b$と同式となる(2.5節の式$(j)$第2式と式$(m)$第1式)。

$$
\left\{
\begin{array}{ll}
\hat{\theta}_a = \frac{\sqrt{{{R_{12}}'}^2 + {{I_{12}}'}^2}}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \;\cos \Bigl( \omega t + \tan^{-1} \frac{{I_{12}}'}{{R_{12}}'} - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'}+ \pi \Bigr)
\\
\hat{q}_b = \hat{\theta}_{ar} = \frac{1}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \; \cos \Bigl( \omega t - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'} \Bigr)
\end{array}
\right.  \qquad (j)
$$

$$
\left\{
\begin{array}{ll}
A_a \cos \varphi_a = \frac{\sqrt{{{R_{12}}'}^2 + {{I_{12}}'}^2}}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \;\cos \Bigl( \tan^{-1} \frac{{I_{12}}'}{{R_{12}}'} - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'}+ \pi \Bigr) \\
A_a \sin \varphi_a = \frac{\sqrt{{{R_{12}}'}^2 + {{I_{12}}'}^2}}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \;\sin \Bigl( \tan^{-1} \frac{{I_{12}}'}{{R_{12}}'} - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'}+ \pi \Bigr) \\
A_b \cos \varphi_b = \frac{1}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \; \cos \Bigl( - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'} \Bigr) \\
A_b \sin \varphi_b = \frac{1}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \; \sin \Bigl( - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'} \Bigr) \\
\end{array}
\right.  \qquad (j')
$$

#### (5) 有効熱容量の計算式

導出過程は2.5 (7)を参照のこと。

- 室内側($n=1$側)の単位面積あたりの有効熱容量$\kappa_1 \:[J/(m^2･K)]$：

  - 有効熱容量は、$k$に関わらず、「表面熱伝達層を含む」構成で求めることとする。
    - 行列の要素から求める場合は、$\textbf{Z}$を使用：式$(22')$
    - 周波数応答の係数から求める場合は、式$(g')$の係数を使用：式$(n)$

$$\kappa_1 = \frac{T}{2 \pi} \Biggl| \frac{Z_{11}-1}{Z_{12}} \Biggr| = \frac{1}{\omega} \Biggl| \frac{Z_{11}-1}{Z_{12}} \Biggr| \qquad (22')$$

$$ \kappa_a = \frac{1}{\omega} \sqrt { (A_a \sin \varphi_a  - A_b \sin \varphi_b) ^ 2 + (A_a \cos \varphi_a - A_b \cos \varphi_b ) ^ 2 } \qquad (n) \\ $$ 

#### (6) 熱貫流率、熱容量、単位応答の定常項の計算式

- 熱貫流率$U \; [W/(m^2･K)]$：

$$ U = 1 \bigg/ \sum_{n=1}^{N} \frac{ d_n }{ \lambda_n } \qquad (p)$$

- 躯体構成全体の表面積$1m^2$あたりの熱容量$C \; [kJ/(m^2･K)]$：

$$ C = \sum_{n=1}^{N} (c\rho)_n d_n \ \qquad (q)$$

- 【$k=1$】単位応答の定常項

$$ B_{0a}=B_{0b}=U=1 \bigg/ \sum_{n=1}^{N} \frac{ d_n }{ \lambda_n } \qquad (r')$$

- 【$k=2$】単位応答の定常項

$$ B_{0a} = \sum_{n=2}^{N} \frac{ d_n }{ \lambda_n } \qquad (s')$$
$$ B_{0b} = 1 \qquad (t)$$

### 2.3 計算プログラム

#### (1) 一次元壁体の構成＋熱物性から周波数応答を算出するモジュール `calc_frequencyresponse`

In [37]:
""" 2章の本体プログラム --- 2.2 (1)～(3)を受けて、2.2 (4)(5) を計算するモジュール """
def calc_frequencyresponse(kk, d_n, lambda_n, c_rho_n, omega):

    ZZ, ZZ_2_N = calc_z(d_n, lambda_n, c_rho_n, omega)

    if kk==1:   #n=1表面温度励振、n=N温度固定→n=1吸熱応答、n=N貫流応答
        """ 2.2 式(g)第1式の計算 """
        rr_a = abs(ZZ[0,0]) / abs(ZZ[0,1])                                      #振幅    2.2 式(g)第1式
        pp_a = cmath.phase(ZZ[0,0]) - cmath.phase(ZZ[0,1]) + cmath.pi           #位相差  2.2 式(g)第1式
        """ 2.2 式(g)第2式の計算 """
        rr_b = 1 / abs(ZZ[0,1])                                                 #振幅    2.2 式(g)第2式
        pp_b = - cmath.phase(ZZ[0,1]) - cmath.pi                                #位相差  2.2 式(g)第2式
    elif kk==2:   #n=2表面熱流励振、n=N温度固定→n=2表面温度応答、n=N熱流応答
        """ 2.2 式(j)第1式の計算 """        
        rr_a = abs(ZZ_2_N[0,1]) / abs(ZZ_2_N[0,0])                              #振幅    2.2 式(j)第1式
        pp_a = cmath.phase(ZZ_2_N[0,1]) - cmath.phase(ZZ_2_N[0,0]) + cmath.pi   #位相差  2.2 式(j)第1式
        """ 2.2 式(j)第2式の計算 """ 
        rr_b = 1 / abs(ZZ_2_N[0,0])                                             #振幅    2.2 式(j)第1式
        pp_b = - cmath.phase(ZZ_2_N[0,0])                                       #位相差  2.2 式(j)第2式

    AC_a = rr_a * np.cos(pp_a)                                                  #a側応答のAcosφ
    AS_a = rr_a * np.sin(pp_a)                                                  #a側応答のAsinφ
    AC_b = rr_b * np.cos(pp_b)                                                  #b側応答のAcosφ
    AS_b = rr_b * np.sin(pp_b)                                                  #b側応答のAsinφ
    
    """ 2.2 式(22')の計算：有効熱容量の算定 """ 
    kappa_1 = 1 / omega * abs( ( ZZ[0,0] - 1 ) / ZZ[0,1] ) / 1000               #室内側有効熱容量κ1[kJ/(m3k)]
#    """ 2.2 式(n)の計算：有効熱容量の算定 """ 
#    if kk==1: 
#        #室内側有効熱容量κ1[kJ/(m3k)]
#        kappa_10 = 1 / omega * ( (AS_a - AS_b )**2 + (AC_a - AC_b )**2 )**0.5 / 1000
#    elif kk==2:  
#        kappa_10 = -1
    
    return [AC_a, AS_a, AC_b, AS_b, kappa_1]


#### (2) 行列作成モジュール `calc_z`

In [36]:
""" 層1～層N、層2～層Nの四端子行列作成 --- 2.2 (3) に対応した計算モジュール """
def calc_z(d_n, lambda_n, c_rho_n, omega):
    ZZ = np.array([[1,0], [0,1]])
    for i in range(len(d_n)-1,-1,-1):
        ZZ = np.dot(ZZ, calc_z_n(d_n[i], lambda_n[i],c_rho_n[i], omega)) # 2×2の行列同士の積  2.2 式(16'')
        if i==1:
            ZZ_2_Nl = ZZ      # 2.2 式(16''') ←n=1表面熱伝達層を含まない
    return [ZZ, ZZ_2_Nl]


#### (3) 各層の行列作成モジュール `calc_z_n`

In [34]:
""" 各層の四端子行列作成 --- 2.2 (1)(2) に対応した計算モジュール """
import cmath
import numpy as np

def calc_z_n(d, lamb, c_rho, omega):
    if c_rho > 0:
        c_rho *= 1000  #面積熱容量cρは[kJ]だてで受取→[J]だてに変換
        delta = ( (2 * lamb) / (omega * c_rho) )**0.5       #2.2 式(11')
        xi = d / delta                                      #2.2 式(a)
        Z_11 = np.cosh(xi) * np.cos(xi) + (np.sinh(xi) * np.sin(xi))*1j
        Z_12 = - delta / (2 * lamb) * (( np.sinh(xi) * np.cos(xi) + np.cosh(xi) * np.sin(xi) ) 
                                     + ( np.cosh(xi) * np.sin(xi) - np.sinh(xi) * np.cos(xi) )*1j)
        Z_21 = - lamb / delta * (( np.sinh(xi) * np.cos(xi) - np.cosh(xi) * np.sin(xi) ) 
                               + ( np.sinh(xi) * np.cos(xi) + np.cosh(xi) * np.sin(xi) )*1j)    #2.2 式(14)
    else:
        Z_11 = 1
        Z_12 = - d / lamb
        Z_21 = 0                                    #2.2 式(15')(18')
    
    Z_n = np.array([[Z_11,Z_12], [Z_21,Z_11]])
    return Z_n


#### (4) 熱貫流率、熱容量、単位応答の定常項作成モジュール `calc_u_c_b0`

In [35]:
""" 熱貫流率、熱容量、単位応答の定常項作成 --- 2.2 (6) に対応した計算モジュール """
def calc_u_c_b0(d_n, lambda_n, c_rho_n):
    RR = 0
    CC = 0
    for i in range(len(d_n)-1,-1,-1):
        RR += d_n[i] / lambda_n[i]  #抵抗の和[m2K/W]
        CC += c_rho_n[i] * d_n[i]   #熱容量の和[kJ/(m2K)], 2.2 式(q)
        if i==1:
            RR_2_N = RR             #n=1表面熱伝達層を含まない熱抵抗
    UU = 1 / RR                     #熱貫流率(n=1表面熱伝達層を含む), 2.2 式(p)

    B0_a = UU                   #a側単位応答の定常項(k=1の場合), 2.2 式(r)
    B0_b = UU                   #b側単位応答の定常項(k=1の場合), 2.2 式(r)

    B0_a_2_Nl = RR_2_N          #a側単位応答の定常項(k=2の場合), 2.2 式(s)
    B0_b_2_Nl = 1               #b側単位応答の定常項(k=2の場合), 2.2 式(t)
 
    return [UU, CC, B0_a, B0_b, B0_a_2_Nl, B0_b_2_Nl]
       

### 2.4 計算プログラムの確認

- セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内


#### (1) 一次元壁体の構成＋熱物性から周波数応答を算出するモジュール `calc_frequencyresponse`

In [38]:
""" 2章の本体プログラム """
""" 入力値(その1, 「高断熱・大熱容量床」のn=1表面熱伝達層を含む場合：熱流応答) """ 
kk = 1                                                          #単位応答の種別
d_n =      [1,     0.13448,  0.012,   0.1,    0.045,  1]        #各層の厚さd[m]
lambda_n = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]      #各層の熱伝導率λ[W/(mK)]
c_rho_n =  [0,     1896.260, 715.806, 13.395, 25.116, 0]        #各層の容積比熱cρ[kJ/(m3K)] 
T = 24                                                          #周期[hour]
omega = 2 * cmath.pi / (T * 3600)                               #角周波数[rad/s]
calc_result2 = calc_frequencyresponse(kk, d_n, lambda_n, c_rho_n, omega)
print ('その1, 「高断熱・大熱容量床」のn=1表面熱伝達層を含む場合の結果')
print ('Acosφ|a={}, Asinφ|a={}'.format(calc_result2[0], calc_result2[1]))
print ('Acosφ|b={}, Asinφ|b={}'.format(calc_result2[2], calc_result2[3]))
print ('Κa={} ←有効熱容量は常にn=1表面熱伝達層を含む場合の値'.format(calc_result2[4]))

#Excelシート「周期定常→指数項二項近似」上の出力値
# Acosφ|a = 5.112460236,  Asinφ|a = 1.591950776                セルR23C6,R23C7
# Acosφ|b = -0.007437852,  Asinφ|b = -0.060418385              セルR23C10,R23C11
# Κa = 73.97939071                                              セルR23C26

""" 入力値(その2, 「高断熱・大熱容量床」のn=1表面熱伝達層を含まない場合：表面温度応答) """ 
kk = 2                                                          #単位応答の種別
calc_result2 = calc_frequencyresponse(kk, d_n, lambda_n, c_rho_n, omega)
print ('その2, 「高断熱・大熱容量床」のn=1表面熱伝達層を含まない場合の結果')
print ('Acosφ|a={}, Asinφ|a={}'.format(calc_result2[0], calc_result2[1]))
print ('Acosφ|b={}, Asinφ|b={}'.format(calc_result2[2], calc_result2[3]))
print ('Κa={} ←有効熱容量は常にn=1表面熱伝達層を含む場合の値'.format(calc_result2[4]))

#Excelシート「周期定常→指数項二項近似」上の出力値
# Acosφ|a = 0.029057487,  Asinφ|a = -0.055523695               セルR57C14,R57C15
# Acosφ|b = -0.004680904,  Asinφ|b = -0.010360299              セルR57C18,R57C19
# Κa = 73.97939071                                              セルR23C26 ←有効熱容量は常にn=1表面熱伝達層を
#                                                                             含む場合の値

その1, 「高断熱・大熱容量床」のn=1表面熱伝達層を含む場合の結果
Acosφ|a=5.112460239934605, Asinφ|a=1.5919508295206453
Acosφ|b=-0.007437778627315811, Asinφ|b=-0.06041840071398695
Κa=73.97939009726596 ←有効熱容量は常にn=1表面熱伝達層を含む場合の値
その2, 「高断熱・大熱容量床」のn=1表面熱伝達層を含まない場合の結果
Acosφ|a=0.029057485360126326, Asinφ|a=-0.055523696228776974
Acosφ|b=-0.004680892284479232, Asinφ|b=-0.01036030558146521
Κa=73.97939009726596 ←有効熱容量は常にn=1表面熱伝達層を含む場合の値


#### (2) 行列作成モジュール `calc_z`

In [39]:
""" 層1～層N、層2～層Nの四端子行列作成 """
""" 入力値(その1,「高断熱・大熱容量床」のZ,ZZ_2_N) """ 
d_n =      [1,     0.13448,  0.012,   0.1,    0.045,  1]        #各層の厚さd[m]
lambda_n = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]      #各層の熱伝導率λ[W/(mK)]
c_rho_n =  [0,     1896.260, 715.806, 13.395, 25.116, 0]        #各層の容積比熱cρ[kJ/(m3K)] 
T = 24                                                          #周期[hour]
omega = 2 * cmath.pi / (T * 3600)                               #角周波数[rad/s]
ZZ, ZZ_2_N = calc_z(d_n, lambda_n, c_rho_n, omega)
print ("その1,「高断熱・大熱容量床」のZ(n=1表面熱伝達層含む)")
print (ZZ)
print ("その1,「高断熱・大熱容量床」のZZ_2_N(n=1表面熱伝達層含まない)")
print (ZZ_2_N)

# 上記(その1,「高断熱・大熱容量床」のZ(n=1表面熱伝達層含む)の結果 """ 
# 「周期定常→指数項二項近似」シート内 セルR23C30～R23C37
# Z_11 = -36.21683125 + 80.15912437j
# Z_12 = 2.007136534 -16.30416337j
# Z_21 = 13.59423945 -15.82926307j
# Z_22 = -1.549787127 + 3.566977279j

# 上記(その1,「高断熱・大熱容量床」のZZ_2_N(n=1表面熱伝達層含まない)の結果 """
# 「周期定常→指数項二項近似」シート内 セルR57C30～R57C37
# Z_11 = -36.21683125 + 80.15912437j
# Z_12 = -3.398360667 - 4.34011496j
# Z_21 = 13.59423945 - 15.82926307j
# Z_22 = 0.479203837 + 1.204400702j

""" 入力値(その2,ISO 13786:2007のp.20 D.2の計算例の確認 """ 
d_n =      [1,      0.2,      0.1,     0.005,    1]             #各層の厚さd[m]
lambda_n = [1/0.13, 1.8,      0.04,    1.0,      1/0.04]        #各層の熱伝導率λ[W/(mK)]
c_rho_n =  [0,      2400,     30*1.4,  1200*1.5, 0]             #各層の容積比熱cρ[kJ/(m3K)] 
T = 24                                                          #周期[hour]
omega = 2 * cmath.pi / (T * 3600)                               #角周波数[rad/s]
ZZ, ZZ_2_N = calc_z(d_n, lambda_n, c_rho_n, omega)
print ('その2,ISO 13786:2007のp.20 D.2の計算例')
print ('|Z11|={}, |Z21|={}, |Z12|={}, |Z21|={}'.format(abs(ZZ[0,0]),abs(ZZ[1,0]),abs(ZZ[0,1]),abs(ZZ[1,1])))
print ('Δh11={}, Δh21={}, Δh12={}, Δh22={}'.format(cmath.phase(ZZ[0,0])*24/(2*cmath.pi)
    ,cmath.phase(ZZ[1,0])*24/(2*cmath.pi),cmath.phase(ZZ[0,1])*24/(2*cmath.pi)
    ,cmath.phase(ZZ[1,1])*24/(2*cmath.pi)))

# 上記(その2,ISO 13786:2007のp.20 D.2の計算例の結果 """
# |Z_11| = 98.12
# |Z_21| = 83.07 W/(m2･K)
# |Z_12| = 16.51 m2K/W
# |Z_22| = 13.99
# Time shift [h]
# Δh_11 = 8.96
# Δh_21 = 0.99
# Δh_12 = -3.89
# Δh_22 = -11.86

その1,「高断熱・大熱容量床」のZ(n=1表面熱伝達層含む)
[[-36.21673012+80.15915949j   2.00711639-16.30416398j]
 [ 13.59420280-15.82928045j  -1.54977957 +3.56697834j]]
その1,「高断熱・大熱容量床」のZZ_2_N(n=1表面熱伝達層含まない)
[[-36.21673012+80.15915949j  -3.39836572 -4.34011032j]
 [ 13.59420280-15.82928045j   0.47920592 +1.20439917j]]
その2,ISO 13786:2007のp.20 D.2の計算例
|Z11|=98.11681926956116, |Z21|=83.0658807888988, |Z12|=16.513090777083594, |Z21|=13.987410086700532
Δh11=8.961826305680894, Δh21=0.9923519741248974, Δh12=-3.8911826439523716, Δh22=-11.858741737804078


#### (3) 各層の行列作成モジュール `calc_z_n`

In [40]:
""" 各層の四端子行列作成 """
""" 入力値(その1, RC13.5mm弱) """ 
d = 0.13448                         #厚さd[m]
lamb = 1.6                          #熱伝導率λ[W/(mK)]
c_rho = 1896.26                     #容積比熱cρ[kJ/(m3K)] 
T = 24                              #周期[hour]
omega = 2 * cmath.pi / (T * 3600)   #角周波数[rad/s]
Z_n = calc_z_n(d, lamb, c_rho, omega)
print ("その1, RC13.5mm弱のZ_n")
print (Z_n)
# === 上記(その1, RC13.5mm弱)の結果
# 「周期定常→指数項二項近似」シート内 セルR57C62～R57C69
# Z_11 = 0.89891674 + 0.774087443j
# Z_12 = -0.082349697-0.021771501j
# Z_21 = 4.803661048-18.16962643j
# Z_22 = 0.89891674+0.774087443j

""" 入力値(その2, 表面熱伝達層) """
d = 1                               #厚さd[m]
lamb = 6.7                          #熱伝導率λ[W/(mK)]
c_rho = 0                           #容積比熱cρ[kJ/(m3K)] 
T = 24                              #周期[hour]
omega = 2 * cmath.pi / (T * 3600)   #角周波数[rad/s]
Z_n = calc_z_n(d, lamb, c_rho, omega)
print ("その2, 表面熱伝達層のZ_n")
print (Z_n)
# === 上記(その2, 表面熱伝達層)の結果
# 「周期定常→指数項二項近似」シート内 セルR23C70～R23C77
# Z_11 = 1
# Z_12 = -0.149253731
# Z_21 = 0
# Z_22 = 1

その1, RC13.5mm弱のZ_n
[[ 0.89891674 +0.77408744j -0.08234970 -0.0217715j ]
 [ 4.80366105-18.16962643j  0.89891674 +0.77408744j]]
その2, 表面熱伝達層のZ_n
[[ 1.         -0.14925373]
 [ 0.          1.        ]]


#### (4) 熱貫流率、熱容量、単位応答の定常項作成モジュール `calc_u_c_b0`

In [41]:
""" 熱貫流率、熱容量、単位応答の定常項作成 """
""" 入力値(その1,「高断熱・大熱容量床」のUU, CC, 定常項) """ 
d_n =      [1,     0.13448,  0.012,   0.1,    0.045,  1]        #各層の厚さd[m]
lambda_n = [6.7,   1.6,      0.16,    0.038,  0.028,  6.7]      #各層の熱伝導率λ[W/(mK)]
c_rho_n =  [0,     1896.260, 715.806, 13.395, 25.116, 0]        #各層の容積比熱cρ[kJ/(m3K)] 
T = 24                                                          #周期[hour]
omega = 2 * cmath.pi / (T * 3600)                               #角周波数[rad/s]
UU, CC, B0_a, B0_b, B0_a_2_Nl, B0_b_2_Nl = calc_u_c_b0(d_n, lambda_n, c_rho_n)
print ("その1,「高断熱・大熱容量床」の結果")
print ('熱貫流率U={},熱容量C={}'.format(UU, CC))
print ('n=1表面熱伝達層含む場合(k=1)のa側励振時の定常項B0_a={},b側定常項B0_b={}'.format(B0_a, B0_b))
print ('n=1表面熱伝達層含まない場合(k=2)のa側励振時の定常項：B0_a_2_Nl={},b側定常項B0_b_2_Nl={}'\
                                                   .format(B0_a_2_Nl, B0_b_2_Nl))
# 上記(その1,「高断熱・大熱容量床」の結果 """ 
# Excelシート「周期定常→指数項二項近似」上の出力値
# U = 0.212934526                                               #R4C8
# B0_a = 0.212934526, B0_b = 0.212934526                        #R84C2,R164C2
# B0_a_2_Nl = 4.547025536, B0_b_2_Nl = 1                        #R244C2,R324C2


その1,「高断熱・大熱容量床」の結果
熱貫流率U=0.21293452605868463,熱容量C=266.0684368
n=1表面熱伝達層含む場合(k=1)のa側励振時の定常項B0_a=0.21293452605868463,b側定常項B0_b=0.21293452605868463
n=1表面熱伝達層含まない場合(k=2)のa側励振時の定常項：B0_a_2_Nl=4.547025535854562,b側定常項B0_b_2_Nl=1


### 2.5 計算式の確認

- ISO 13786, Thermal performance of building components - Dynamic thermal　characteristics - Calculation methods, 2007
- 以下、「小文字アルファベット以外の」式番号は上記文献に対応。ただしダッシュ(')付きは本稿にあわせて変更している。

#### (1) 一次元躯体の構成と熱物性

本稿で扱う一次元躯体は、$N-2\;$層からなる躯体構成要素とその両面の表面熱伝達層を持つ、全体で$N$層で構成される躯体とする。層構成を以下のように割り当てる。

- 層$1 \; (n=1)$：単位応答を計算する側(例：外壁の室内側)の表面熱伝達層
- 層$N \; (n=N)$：反対側(例：外壁の外気側)の表面熱伝達層
- 層$2$～層$N-1 \; (n=2$～$N-1)$：躯体を構成する部材の層(中空層含む)

この層構成は「室内側表面熱伝達層を含まない」単位応答を求める際であっても変えないものとする(室内側表面熱伝達層$\;(n=1)\;$を必ず確保する)。

各層には、以下の物性値等を設定する。

- 厚さ$d_n\;[m]$
- 熱伝導率$\lambda_n \; [W/(m･K)]$
- 容積比熱$(c\rho)_n \; [kJ/(m^3･K)]$

表面熱伝達層や中空層の場合は、$(c\rho)_n=0\;[kJ/(m^3･K)]$とする。

中空層の場合は、厚さ$d_n\;[m]$を、その中空層の熱抵抗$r_n \; [m^2K/W]$で除した値を、$\lambda_n \; [W/(m･K)]$として設定する($\lambda_n \leftarrow d_n / r_n $)。

表面熱伝達層の場合は、$d_n=1\;[m]$とし、表面熱伝達率$\alpha_s \; [W/(m^2･K)]$を$\lambda_n$として設定する($\lambda_n \leftarrow \alpha_s $)。

#### (2) 熱貫流率と熱容量

熱貫流率$U \; [W/(m^2･K)]$は下式で求める。

$$ U = 1 \bigg/ \sum_{n=1}^{N} \frac{ d_n }{ \lambda_n } \qquad (p)$$

躯体構成全体の表面積$1m^2$あたりの熱容量$C \; [kJ/(m^2･K)]$は下式で求める。

$$ C = \sum_{n=1}^{N} (c\rho)_n d_n \ \qquad (q)$$


#### (3) 単位応答の定常項

励振と応答の関係により、求める単位応答の定常項$B_0$は以下の通りとなる。ただし、「室内側表面熱伝達層を含む」応答を求める場合には$a=1$、$b=N$、「室内側表面熱伝達層を含まない」応答を求める場合には$a=2$、$b=N$。

- 温度励振に対する熱流応答

$$ B_0=1 \bigg/ \sum_{n=a}^{b} \frac{ d_n }{ \lambda_n } \qquad (r)$$

- 熱流励振に対する温度応答

$$ B_0=\sum_{n=a}^{b} \frac{ d_n }{ \lambda_n } \qquad (s)$$

- 熱流励振に対する熱流応答、温度励振に対する温度応答

$$ B_0=1 \qquad (t)$$


#### (4) 単層の四端子行列

##### a) 躯体単層の四端子行列

侵入厚さ$\:\delta[m]\:$

$$ \delta = \sqrt {\frac{\lambda T}{\pi c \rho}} = \sqrt {\frac{2 \lambda}{\omega c \rho}} \qquad (11')$$

と、

$$ \xi = \frac{d}{\delta} \qquad (13)$$

より、層厚$\:d[m]\:$と侵入厚さ$\:\delta[m]\:$の比$\:\xi\:$は、

$$ \xi = d \sqrt {\frac{\pi c \rho}{\lambda T}} = d \sqrt {\frac{\omega c \rho}{2 \lambda}} \qquad (a)$$

となる。ただし、ここで、$\lambda$は熱伝導率$\:[W/(m･K)]$、$T$は周期$\:[s]$、$c$は比熱$\:[J/(kg･K)]$、$\rho$は密度$\:[kg/m^3]$、$\omega$は角周波数$[rad/s]$である。

- 本稿内では、以下については、主に入力の際に単位が変わっている場合があるので注意すること。
   - 時間$T$：$[hour]$か$[s]$か
   - 容積比熱$c\rho$：$[J/(m^3･K)]$か$[kJ/(m^3･K)]$か

単層の四端子行列$\;\textbf{Z} = \begin{pmatrix} Z_{11} & Z_{12} \\ Z_{21} & Z_{22} \end{pmatrix}\;$ の要素は、以下で計算される。

$$ \begin{align}
Z_{11} = Z_{22} &= \cosh(\xi) \cos(\xi) + j \sinh(\xi) \sin(\xi) \\ 
Z_{12} &= - \frac{\delta}{2 \lambda} \{ \sinh(\xi) \cos(\xi) + \cosh(\xi) \sin(\xi) + j [ \cosh(\xi) \sin(\xi) - \sinh(\xi) \cos(\xi) ] \} \\
Z_{21} &= - \frac{\lambda}{\delta} \{\sinh(\xi) \cos(\xi) - \cosh(\xi) \sin(\xi) + j [\sinh(\xi) \cos(\xi) + \cosh(\xi) \sin(\xi)]\}  \qquad (14) \\
\end{align} $$

##### b) 表面熱伝達層、中空層の四端子行列

比熱の影響を無視できる表面熱伝達層や中空層における熱抵抗を$R$とすれば、表面熱伝達層、中空層の単層の四端子行列は以下となる。

$$\;\textbf{Z} = \begin{pmatrix} 1 & -R \\ 0 & 1 \end{pmatrix}  \qquad (15'), \; (18') $$

#### (5) 躯体全体の四端子行列

$N-2\;$層からなる躯体構成要素とその両面に表面熱伝達層がある(全体で$N$層の)構成における割り当ては以下の通りとする。「室内側表面熱伝達層を含まない」単位応答を求める際であっても、室内側表面熱伝達層$\;(n=1)\;$の確保と熱物性の記入は必須とする。

- 層$1$：単位応答を計算する側(例：外壁の室内側)の表面熱伝達層
- 層$N$：反対側(例：外壁の外気側)の表面熱伝達層
- 層$2$～$N-1$：躯体を構成する部材の層(中空層含む)

層$a$～$b$の四端子行列$\;\textbf{Z}\;$は、以下の式で求まる。

$$\textbf{Z} = \begin{pmatrix} Z_{11} & Z_{12} \\ Z_{21} & Z_{22} \end{pmatrix}=\textbf{Z}_b \; \textbf{Z}_{b-1} \;  \cdots \;  \textbf{Z}_{a+1} \;  \textbf{Z}_{a}  \qquad (16') $$

「室内側表面熱伝達層を含む」単位応答を求める場合には$a=1$、$b=N$となる。一方、「室内側表面熱伝達層を含まない」単位応答を求める場合には$a=2$、$b=N$となり、$\textbf{Z}_1$は除外されることに注意する。

式$(16')$の行列内の要素は式$(14)$のように複素数となることから、式$(b)$の乗算を式$(16')$に順次適用することで求めていく($R_{**}$は$Z_{**}$の実数部、$jI_{**}$は虚数部($I_{**}$自体は実数))。

$$\begin{align} \textbf{Z}'' &= \textbf{Z}\;\textbf{Z}' \\
&= \begin{pmatrix} Z_{11} & Z_{12} \\ Z_{21} & Z_{22} \end{pmatrix} \begin{pmatrix} {Z_{11}}' & {Z_{12}}' \\ {Z_{21}}' & {Z_{22}}' \end{pmatrix}
=\begin{pmatrix} R_{11}+jI_{11} & R_{12}+jI_{12} \\ R_{21}+jI_{21} & R_{22}+jI_{22} \end{pmatrix} \begin{pmatrix} {R_{11}}'+j{I_{11}}' & {R_{12}}'+j{I_{12}}' \\ {R_{21}}'+j{I_{21}}' & {R_{22}}'+j{I_{22}}' \end{pmatrix} \\
&= \begin{pmatrix} {Z_{11}}'' & {Z_{12}}'' \\ {Z_{21}}'' & {Z_{22}}'' \end{pmatrix}  \hspace{14cm} (b) \end{align} \\
$$

$$\left\{ \begin{array}{ll}
{Z_{11}}''=(R_{11}{R_{11}}'-I_{11}{I_{11}}'+R_{12}{R_{21}}'-I_{12}{I_{21}}')+j(R_{11}{I_{11}}'+I_{11}{R_{11}}'+R_{12}{I_{21}}'+I_{12}{R_{21}}') \\
{Z_{12}}''=(R_{11}{R_{12}}'-I_{11}{I_{12}}'+R_{12}{R_{22}}'-I_{12}{I_{22}}')+j(R_{11}{I_{12}}'+I_{11}{R_{12}}'+R_{12}{I_{22}}'+I_{12}{R_{22}}') \\
{Z_{21}}''=(R_{21}{R_{11}}'-I_{21}{I_{11}}'+R_{22}{R_{21}}'-I_{22}{I_{21}}')+j(R_{21}{I_{11}}'+I_{21}{R_{11}}'+R_{22}{I_{21}}'+I_{22}{R_{21}}') \\
{Z_{22}}''=(R_{21}{R_{12}}'-I_{21}{I_{12}}'+R_{22}{R_{22}}'-I_{22}{I_{22}}')+j(R_{21}{I_{12}}'+I_{21}{R_{12}}'+R_{22}{I_{22}}'+I_{22}{R_{22}}')
\end{array} \right.   \\ $$

#### (6) 温度と熱流の周波数応答

##### a) 温度と熱流の関係式

$a$側の温度変動$\hat{\theta}_a$、熱流変動$\hat{q}_a$、$b$側の温度変動$\hat{\theta}_b$、熱流変動$\hat{q}_b$とし、$a \rightarrow b$の熱流の向きを正とすると、式$(16')$の四端子行列を用いて、

$$\begin{pmatrix} \hat{\theta}_b \\ - \hat{q}_b \end{pmatrix}=\begin{pmatrix} Z_{11} & Z_{12} \\ Z_{21} & Z_{22} \end{pmatrix} \begin{pmatrix} \hat{\theta}_a \\ \hat{q}_a \end{pmatrix}  \qquad (12') \\ $$

の関係をみたす。これを書き直すと、以下の式を得る。

$$\begin{pmatrix} \hat{q}_a \\ - \hat{q}_b \end{pmatrix} = \frac{1}{Z_{12}} \begin{pmatrix} - Z_{11} & 1 \\ 1 & - Z_{22} \end{pmatrix} \begin{pmatrix} \hat{\theta}_a \\ \hat{\theta}_b \end{pmatrix}  \qquad (B.8') \\ $$

$\hat{\theta}_a = |{\theta}_a|e^{ j (\omega t + \psi_a ) }$、$\hat{\theta}_b = |{\theta}_b|e^{ j (\omega t + \psi_b ) }$、$\hat{q}_a = |{q}_a|e^{ j (\omega t + \varphi_a ) }$、$\hat{q}_b = |{q}_b|e^{ j (\omega t + \varphi_b ) }$ とすると、式$(12')$から、

$$\begin{pmatrix} |{\theta}_b|e^{ j (\omega t + \psi_b ) } \\ - |{q}_b|e^{ j (\omega t + \varphi_b ) } \end{pmatrix}=\begin{pmatrix} R_{11}+jI_{11} & R_{12}+jI_{12} \\ R_{21}+jI_{21} & R_{22}+jI_{22} \end{pmatrix} \begin{pmatrix} |{\theta}_a|e^{ j (\omega t + \psi_a ) } \\ |{q}_a|e^{ j (\omega t + \varphi_a ) } \end{pmatrix} \qquad (c) \\ $$

が、式$(B.8')$から、

$$\begin{pmatrix} |{q}_a|e^{ j (\omega t + \varphi_a ) } \\ - |{q}_b|e^{ j (\omega t + \varphi_b ) } \end{pmatrix} = \frac{1}{R_{12}+jI_{12}} \begin{pmatrix} - (R_{11}+jI_{11}) & 1 \\ 1 & - (R_{22}+jI_{22}) \end{pmatrix} \begin{pmatrix} |{\theta}_a|e^{ j (\omega t + \psi_a ) } \\ |{\theta}_b|e^{ j (\omega t + \psi_b ) } \end{pmatrix}  \qquad (d) \\ $$

が全く同じ式として得られる。

##### b) 温度励振時の熱流応答

$\hat{\theta}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$として、$a$側での単位振幅の温度励振の場合を考えると、$|{\theta}_a|=1$、$\psi_a=0$、$|{\theta}_b|=0$となり、式$(c)$、$(d)$の実数部のみ考慮すれば良いことになる。式$(c)$第1式から、

$$\begin{align}
& -(R_{11} \cos \omega t - I_{11} \sin \omega t )=|{q}_a| \{ R_{12} \cos (\omega t + \varphi_a ) - I_{12} \sin (\omega t + \varphi_a ) \} \\
\Rightarrow \quad &\sqrt{{R_{11}}^2 + {I_{11}}^2} \cos(\omega t + {\psi_a}' + \pi) = |{q}_a|\sqrt{{R_{12}}^2 + {I_{12}}^2} \cos (\omega t + \varphi_a + {\varphi_a}') \\
\Rightarrow \quad &|{q}_a| = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}}, \qquad \varphi_a = {\psi_a}' - {\varphi_a}' + \pi = \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi  \qquad (e) \\
\end{align} $$

を得る。また、式$(d)$第2式から、

$$ \begin{align} 
& - |{q}_b| \cos (\omega t + \varphi_b) = \frac{R_{12}}{{R_{12}}^2 + {I_{12}}^2}\cos \omega t + \frac{I_{12}}{{R_{12}}^2 + {I_{12}}^2}\sin \omega t \\
\Rightarrow \quad & |{q}_b| \cos (\omega t + \varphi_b + \pi) = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\cos \Bigl( \omega t - \tan^{-1} \frac{I_{12}}{R_{12}} \Bigr) \\ 
\Rightarrow \quad & |{q}_b| = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}}, \qquad \varphi_b = - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi  \qquad (f)\\
\end{align} $$ 

を得る。よって、$\hat{\theta}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$の$a$側での単位温度励振の場合の熱流応答は、

$$
\left\{
\begin{array}{ll}
\hat{q}_a = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\cos \Bigl( \omega t +  \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi \Bigr)
\\
\hat{q}_b = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \; \cos \Bigl( \omega t - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi \Bigr)
\end{array}
\right.  \qquad (g)
$$

で求まることになる。

##### c) 表面熱流励振時の表面温度応答＋裏面熱流応答

$\hat{q}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$として、$a$側で熱流の単位振幅励振、$b$側で温度固定の場合を考えると、$|{q}_a|=1$、$\varphi_a=0$、$|{\theta}_b|=0$となり、同様に式$(c)$、$(d)$の実数部のみ考慮する。式$(c)$第1式から、

$$\begin{align}
& - ( R_{12} \cos \omega t - I_{12} \sin \omega t ) = |{\theta}_a| \{ R_{11} \cos (\omega t + \psi_a ) - I_{11} \sin (\omega t + \psi_a )\} \\
\Rightarrow \quad &\sqrt{{R_{12}}^2 + {I_{12}}^2} \cos(\omega t + {\varphi_a}' + \pi) = |{\theta}_a| \sqrt{{R_{11}}^2 + {I_{11}}^2} \cos (\omega t + \psi_a + {\psi_a}') \\
\Rightarrow \quad &|{\theta}_a| = \frac{\sqrt{{R_{12}}^2 + {I_{12}}^2}}{\sqrt{{R_{11}}^2 + {I_{11}}^2}}, \qquad 
\psi_a = {\varphi_a}' + \pi - {\psi_a}' =  \tan^{-1} \frac{I_{12}}{R_{12}} - \tan^{-1} \frac{I_{11}}{R_{11}}+ \pi \qquad (h) \\
\end{align} $$

を得る。また、式$(d)$第2式から、

$$ \begin{align} 
& - |{q}_b| \cos (\omega t + \varphi_b) = \frac{R_{12}}{{R_{12}}^2 + {I_{12}}^2} |{\theta}_a| \cos (\omega t + \psi_a) + \frac{I_{12}}{{R_{12}}^2 + {I_{12}}^2} |{\theta}_a| \sin (\omega t + \psi_a) \\
\Rightarrow \quad & |{q}_b| \cos (\omega t + \varphi_b + \pi) = \frac{|{\theta}_a|}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\cos \Bigl( \omega t + \psi_a - \tan^{-1} \frac{I_{12}}{R_{12}} \Bigr) \\ 
\Rightarrow \quad & |{q}_b| = \frac{|{\theta}_a|}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} = \frac{1}{\sqrt{{R_{11}}^2 + {I_{11}}^2}}, \qquad \varphi_b = \psi_a - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi =  - \tan^{-1} \frac{I_{11}}{R_{11}} \qquad (i)\\
\end{align} $$ 

を得る。よって、$\hat{q}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$と、$a$側で熱流の単位振幅励振、$b$側で温度固定とする場合の$a$側温度応答、$b$側熱流応答は、

$$
\left\{
\begin{array}{ll}
\hat{\theta}_a = \frac{\sqrt{{R_{12}}^2 + {I_{12}}^2}}{\sqrt{{R_{11}}^2 + {I_{11}}^2}} \;\cos \Bigl( \omega t + \tan^{-1} \frac{I_{12}}{R_{12}} - \tan^{-1} \frac{I_{11}}{R_{11}}+ \pi \Bigr)
\\
\hat{q}_b = \frac{1}{\sqrt{{R_{11}}^2 + {I_{11}}^2}} \; \cos \Bigl( \omega t - \tan^{-1} \frac{I_{11}}{R_{11}} \Bigr)
\end{array}
\right.  \qquad (j)
$$

で求まることになる。

##### d) 裏面温度励振、表面熱流固定時の表面温度応答＋裏面熱流応答

【貫流応答の相反則成立確認のための参考】

$\hat{q}_a = 0$、$\hat{\theta}_b =  \cos \omega t \:$として、$a$側で熱流固定、$b$側で温度の単位振幅励振の場合を考えると、$|{q}_a|=0$、$|{\theta}_b|=1$、$\psi_b=0$となり、同様に式$(c)$の実数部のみ考慮する。

式$(c)$第1式から、

$$\begin{align}
& \cos \omega t = |{\theta}_a| \{ R_{11} \cos (\omega t + \psi_a ) - I_{11} \sin (\omega t + \psi_a )\} \\
\Rightarrow \quad & \cos \omega t = |{\theta}_a| \sqrt{{R_{11}}^2 + {I_{11}}^2} \cos \Bigl( \omega t + \psi_a + \tan^{-1} \frac{I_{11}}{R_{11}} \Bigr) \\
\Rightarrow \quad & |{\theta}_a| = \frac{1}{\sqrt{{R_{11}}^2 + {I_{11}}^2}}, \qquad 
\psi_a = - \tan^{-1} \frac{I_{11}}{R_{11}} \qquad (k) \\
\end{align} $$

を得る。また、式$(c)$第2式から、

$$ \begin{align} 
& - |{q}_b| \cos (\omega t + \varphi_b) = |{\theta}_a| \{R_{21} \cos (\omega t + \psi_a) - I_{21} \sin (\omega t + \psi_a) \} \\
\Rightarrow \quad & |{q}_b| \cos (\omega t + \varphi_b + \pi) = |{\theta}_a| \sqrt{{R_{21}}^2 + {I_{21}}^2} \;\cos \Bigl( \omega t + \psi_a + \tan^{-1} \frac{I_{21}}{R_{21}} \Bigr) \\ 
\Rightarrow \quad & |{q}_b| = |{\theta}_a| \sqrt{{R_{21}}^2 + {I_{21}}^2} = \frac{\sqrt{{R_{21}}^2 + {I_{21}}^2}}{\sqrt{{R_{11}}^2 + {I_{11}}^2}}, \qquad \varphi_b = \psi_a + \tan^{-1} \frac{I_{21}}{R_{21}} -  \pi =  \tan^{-1} \frac{I_{21}}{R_{21}} - \tan^{-1} \frac{I_{11}}{R_{11}} -  \pi  \qquad (l) \\
\end{align} $$ 

を得る。よって、$\hat{q}_a = 0$、$\hat{\theta}_b =  \cos \omega t \:$と、$a$側で熱流固定、$b$側で温度の単位振幅励振とする場合の$a$側温度応答、$b$側熱流応答は、

$$
\left\{
\begin{array}{ll}
\hat{\theta}_a = \frac{1}{\sqrt{{R_{11}}^2 + {I_{11}}^2}} \;\cos \Bigl( \omega t - \tan^{-1} \frac{I_{11}}{R_{11}} \Bigr)
\\
\hat{q}_b =  \frac{\sqrt{{R_{21}}^2 + {I_{21}}^2}}{\sqrt{{R_{11}}^2 + {I_{11}}^2}} \; \cos \Bigl( \omega t +  \tan^{-1} \frac{I_{21}}{R_{21}} - \tan^{-1} \frac{I_{11}}{R_{11}} -  \pi \Bigr)
\end{array}
\right.  \qquad (m)
$$

で求まることになり、式$(j)$第2式と式$(m)$第1式が等しくなることから相反則が成り立つことが確認できる。


#### (7) 有効熱容量

室内側($a$側)の単位面積あたりの有効熱容量$\kappa_a \:[J/(m^2･K)]$は、下式で求められる。

$$\kappa_a = \frac{T}{2 \pi} \Biggl| \frac{Z_{11}-1}{Z_{12}} \Biggr| = \frac{1}{\omega} \Biggl| \frac{Z_{11}-1}{Z_{12}} \Biggr| \qquad (22')$$

有効熱容量$\kappa_a$の意味するところは、$a$側が単位振幅で温度振動するときの$a$側熱流応答と$b$側熱流応答の差の時間積分(すなわち、躯体の蓄熱量の時間変化)の振幅になり、式$(g)$の$\hat{q}_a$、$\hat{q}_b$から下式で求めることもできる。

$$ \begin{align} 
\kappa_a &= \Bigl| \int (\hat{q}_a - \hat{q}_b) \;dt \; \Bigr| \\
&= \Bigl| \int \{ |{q}_a| \cos (\omega t + \varphi_a ) - |{q}_b| \cos (\omega t + \varphi_b ) \} dt \; \Bigr| \\
&= \frac{1}{\omega} \; \Bigl| \; |{q}_a| \sin (\omega t + \varphi_a ) - |{q}_b| \sin (\omega t + \varphi_b ) \;\Bigr| \\
&= \frac{1}{\omega} \; \Bigl| \; (|{q}_a| \sin \varphi_a  - |{q}_b| \sin \varphi_b) \cos \omega t + (|{q}_a| \cos \varphi_a - |{q}_b| \cos \varphi_b ) \sin \omega t \;\Bigr| \\
&= \frac{1}{\omega} \; \Biggl| \sqrt { (|{q}_a| \sin \varphi_a  - |{q}_b| \sin \varphi_b) ^ 2 + (|{q}_a| \cos \varphi_a - |{q}_b| \cos \varphi_b ) ^ 2 } \cos \Biggl( \omega t - tan ^ {-1} \frac { |{q}_a| \cos \varphi_a - |{q}_b| \cos \varphi_b }{ |{q}_a| \sin \varphi_a  - |{q}_b| \sin \varphi_b } \Biggr) \;\Biggr| \\
&= \frac{1}{\omega} \sqrt { (|{q}_a| \sin \varphi_a  - |{q}_b| \sin \varphi_b) ^ 2 + (|{q}_a| \cos \varphi_a - |{q}_b| \cos \varphi_b ) ^ 2 } \qquad (n) \\
\end{align} $$ 

有効熱容量$\kappa_a$は、周期$T$によって値が変わる。










## 3. 二組の角周波数に対応した周波数応答から指数項二項で近似した単位応答を算出するモジュール


ある(一次元)躯体構成において二組の角周波数に対応して定まる周波数応答二組から、指数項二項で近似した単位応答を算出する。周波数応答が熱流応答の場合には熱流の単位応答が、表面温度の周波数応答からは表面温度の単位応答が求まる。


### 3.1 入力値と出力値


#### (1) 入力値

- Betaの収束計算の初期値：$\beta_{1\_0}$
  - ↑ デフォルトでは$\beta_{1\_0} = 10^{-10}$とするが、収束の状況によって異なる値を設定する。
  
  
- 単位応答の定常項$([W/(m ^ 2K)].or.[m ^ 2K/W].or.[-])$：$B_0$
- 角周波数二組$[rad/s]$：$\omega_1$、$\omega_2$
- 対応する周波数応答の係数二組$([W/(m ^ 2K)].or.[m ^ 2K/W].or.[-])$：$A_1 \cos \varphi_1$、$A_1 \sin \varphi_1$、$A_2 \cos \varphi_2$、$A_2 \sin \varphi_2$
  - ↑ 角周波数$\omega$における周波数応答 $H_p = A_p \cos (\omega_p t + \varphi_p) = A_p \cos \varphi_p \cos \omega_p t - A_p \sin \varphi_p \sin \omega_p t \quad ( p = 1,\! 2 )$ における


#### (2) 引数と入力値

- `calc_unitresponse(initBeta_1, B_0, omega_1, AC_1, S_1, omega_2, AC_2, S_2)`において、
  
  - `initBeta_1` $ = \beta_{1\_0}$  
  - `B_0` $ = B_0$
  - `omega_1` $ = \omega_1$
  - `AC_1` $ = A_1 \cos \varphi_1$
  - `S_1` $ = A_1 \sin \varphi_1$
  - `omega_2` $ = \omega_2$
  - `AC_2` $ = A_2 \cos \varphi_2$
  - `S_2` $ = A_2 \sin \varphi_2$
   

#### (3) 出力値

- 係数$([W/(m ^ 2K)].or.[m ^ 2K/W].or.[-])$：$B_1$、$B_2$
- 時定数$[1/sec]$：$\beta_1$、$\beta_2$
  - ↑ 指数項二項からなる単位応答：$h = B_0 + B_1 e ^ {- \beta_1 t} + B_2 e ^ {- \beta_2 t}$ における


#### (4) 戻り値と出力値

- `B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2 = calc_unitresponse(initBeta_1, B_0, omega_1, AC_1, S_1, omega_2, AC_2, S_2)`   において、
  
  - `B_1` $ = B_1$
  - `B_2` $ = B_2$    
  - `Beta_1` $ = \beta_1$  
  - `Beta_2` $ = \beta_2$ 
  - 以下制御・確認用戻り値
    - `i` :  $\beta_m$の収束に要した計算回数。`i > 0` で計算成功、`i <= 0` で計算失敗
    - `Diff_Beta_1` :  $\beta_1$の収束判定時の残差
    - `Diff_Beta_2` :  $\beta_2$の収束判定時の残差
    

### 3.2 計算の流れ

導出過程は3.5 (1)(2)を参照のこと。

- 式$(8)$で$\beta_1$、$\beta_2$を収束計算から求める ($\beta_1 < \beta_2$、$\beta_m > 0$)。
- 本稿内での式$(8)$の収束計算の手順は以下の通り。
   - $\beta_1$の初期値は$10 ^ {-10}$。収束の状況によって異なる値を設定できる。
   - 式$(8)$で、$\beta_1$から$\beta_2$を、$\beta_2$から$\beta_1$を計算して1ステップ。
   - $\beta_1$、$\beta_2$の双方について、前ステップの値との差が$10 ^ {-10}$を下回れば収束計算終了。
   - 計算収束後に$\beta_1 < \beta_2$となるように大小関係を整える。
   - 計算収束後に$\beta_1 = \beta_2$となる場合は計算失敗→二組の角周波数$\omega$を再選定する必要がある。
   - 計算収束後に$\beta_m \leq 0$となる場合も計算失敗→二組の角周波数$\omega$を再選定する必要がある。


- $\beta_1$、$\beta_2$が定まった後、式$(9)$で$B_1$、$B_2$を求める。
- 西澤、三浦：周波数特性から単位応答を近似的に求める方法に関する検討, 日本建築学会大会学術講演梗概集D2, pp.586-587, 2017
- 式番号は上記文献に対応

$$ \qquad
C_p = A_p \cos \varphi_p - B_0 = \sum_{m=1}^M \frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2}, \quad
S_p = A_p \sin \varphi_p = \sum_{m=1}^M \frac{B_m \beta_m \omega_p}{{\beta_m}^2 + {\omega_p}^2} \quad ( p = 1,\! 2 )
\qquad (4),\;(5)
\\
$$
$$ \qquad 
\beta_m
= \frac{\omega_p {\omega_q}^2 S_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 \omega_q S_q ( {\beta_n}^2 + {\omega_q}^2 )}
{{\omega_q}^2 C_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 C_q ( {\beta_n}^2 + {\omega_q}^2 )}
, \quad
( m = 1,\! 2,\; n = 3 - m, \; p = 1, \; q = 2)
\qquad (8)
\\
$$
$$ \qquad 
B_m
= \frac{{\beta_m}^2 + {\omega_p}^2}{{\omega_p}^2 ( \beta_n - \beta_m )} ( \beta_n C_p - \omega_p S_p )
, \quad
( m = 1,\! 2,\; n = 3 - m, \; p = 1,\! 2)
\qquad (9)
\\
$$








### 3.3 計算プログラム

#### (1) 3章の本体プログラム＋モジュール

In [42]:
""" 3章の本体プログラム """
""" 二組の角周波数に対応した周波数応答から指数項二項で近似した単位応答を算出するモジュール """
def calc_unitresponse(initBeta_1, B_0, omega_1, AC_1, S_1, omega_2, AC_2, S_2):
    
    C_1 = AC_1 - B_0   #3.2 式(4)
    C_2 = AC_2 - B_0   #3.2 式(4)

    """ 3.2 式(8)の収束計算 """
    i = 0
    Diff_Beta_1 = 999
    Diff_Beta_2 = 999
    Beta_1 = initBeta_1
    Beta_2 = 1

    while (i <= 1000) & (Diff_Beta_1 > 10**(-10)) | (Diff_Beta_2 > 10**(-10)):
        i += 1
        Beta_1p = Beta_1
        Beta_2p = Beta_2
        Beta_2 = calc_beta_m(omega_1, C_1, S_1, omega_2, C_2, S_2, Beta_1)
        Beta_1 = calc_beta_m(omega_1, C_1, S_1, omega_2, C_2, S_2, Beta_2)
        Diff_Beta_1 = abs(Beta_1 - Beta_1p)
        Diff_Beta_2 = abs(Beta_2 - Beta_2p)        
        if (i > 2) & ((Beta_1 <= 0) | (Beta_2 <= 0)):
            i = -i      #i < 0：計算失敗
            break
        if (i > 2) & ((Beta_1 / Beta_2 < 1.001) and (Beta_1 / Beta_2 > 0.999)): 
            i = -i      #i < 0：計算失敗
            break
    
    if (Beta_1 > Beta_2):
        Beta_1, Beta_2 = Beta_2, Beta_1     #β1 < β2
    
    """ 3.2 式(9)の計算 """
    if (Beta_1 == -999):
        B_1 = 0
        B_2 = 0
    else:
        B_1 = calc_b_m(omega_1, C_1, S_1, Beta_1, Beta_2)
            #B_1 = calc_b_m(omega_2, C_2, S_2, Beta_1, Beta_2) でも求まる
        B_2 = calc_b_m(omega_1, C_1, S_1, Beta_2, Beta_1)
            #B_2 = calc_b_m(omega_2, C_2, S_2, Beta_2, Beta_1) でも求まる  
    
    Calc_Result1 = [B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2]

    return Calc_Result1


In [43]:
""" 3.2 式(8)の関数 """
def calc_beta_m(omega_p, C_p, S_p, omega_q, C_q, S_q, beta_n):
    beta_m = (( omega_p * omega_q**2 * S_p * ( beta_n**2 + omega_p**2 )
              - omega_p**2 * omega_q * S_q * ( beta_n**2 + omega_q**2 ))
            / ( omega_q**2 * C_p * ( beta_n**2 + omega_p**2 )
              - omega_p**2 * C_q * ( beta_n**2 + omega_q**2 )))
    return beta_m

In [44]:
""" 3.2 式(9)の関数 """
def calc_b_m(omega_p, C_p, S_p, beta_m, beta_n):
    B_m = (( beta_m**2 + omega_p**2 ) 
           / ( omega_p**2 * ( beta_n - beta_m )) 
           * ( beta_n * C_p - omega_p * S_p ))
    return B_m

In [45]:
""" 3.2 式(4)の関数：単位応答から周波数応答の Acosφ を算出 """
def calc_ac_p0(omega_p, B_0, B, Beta):     #B, Betaは指数項数分のリスト
    AC_p = B_0 
    for i in range(len(B)):
        AC_p += ( B[i] * omega_p**2 ) / ( Beta[i]**2 + omega_p**2 ) 
    return AC_p

""" 3.2 式(5)の関数：単位応答から周波数応答の Asinφ を算出 """
def calc_as_p0(omega_p, B, Beta):     #B, Betaは指数項数分のリスト
    AS_p = 0
    for i in range(len(B)):
        AS_p += ( B[i] * Beta[i] * omega_p ) / ( Beta[i]**2 + omega_p**2 )
    return AS_p

#### (2) 単位応答から復号した周波数応答と精算値の誤差を確認するための誤算算定モジュール

In [46]:
""" 二つの三角関数間の誤差算定モジュール `calc_rmse` """ 
def calc_rmse(AC_1, AS_1, AC_2, AS_2):
    
    RMSE = ( ( ( AC_1 - AC_2 )**2 + ( AS_1 - AS_2 )**2 ) / 2 )**0.5   #3.5 式(k)
    
    return RMSE

### 3.4 計算プログラムの確認

- セルの参照先は、\周波数応答-単位応答変換(高断熱大熱容量床).xlsm 内の「周期定常→指数項二項近似」シート内


#### (1) 3章の本体プログラムの確認

In [47]:
""" 【計算例】単位温度励振時の吸熱応答 """
""" セルの参照は「周期定常→指数項二項近似」シート内 """
#入力値
initBeta_1 = 10**(-10)                       #Betaの収束計算の初期値
B_00 = 0.212934526                           #定常項(熱貫流率)、セルR84C2
T_01 = 2                                     #一組目の周期[hour]、セルR114C3
omega_01 = 2 * cmath.pi / (T_01 * 3600)      #角周波数[rad/s]、セルR114C4
AC_01 = 5.876837308 + B_00                   #AcosφをC1(セルR114C7)と定常項から算出
S_01 = 0.515005366                           #Asinφ、セルR114C8
T_02 = 24                                    #二組目の周期[hour]、セルR114C9
omega_02 = 2 * cmath.pi / (T_02 * 3600)      #角周波数[rad/s]、セルR114C10
AC_02 = 4.89952571 + B_00                    #AcosφをC1(セルR114C13)と定常項から算出
S_02 = 1.591950776                           #Asinφ、セルR114C14

#Excelシート上の出力値
# B1 = 5.348575591                            セルR114C18
# B2 = 0.801010388                            セルR114C19
# Beta_1 = 2.23046E-05                        セルR114C16
# Beta_2 = 0.000620977                        セルR114C17

B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2 = calc_unitresponse(initBeta_1, B_00, omega_01, AC_01, S_01
                                                                          , omega_02, AC_02, S_02)
print (B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2)
print ('βmの収束計算に{0}回。最後のβ1の残差が{1}。β2の残差が{2}。'.format(abs(i),Diff_Beta_1,Diff_Beta_2))
PlusMinus00 = " + " if B_1 >= 0 else " - "
PlusMinus01 = " + " if B_2 >= 0 else " - "
OK_Not = "" if i >= 0 else "計算失敗 \n"
print ('計算結果：{}h(t) = {}{}{} * exp(-{} * t){}{} * exp(-{} * t)'
       .format(OK_Not,B_00,PlusMinus00,abs(B_1),Beta_1,PlusMinus01,abs(B_2),Beta_2))
    
""" 周波数応答への復号確認 """
print('A1*cosφ1 = {0} = {1}     ←左：入力値、右：単位応答からの計算値。一致してればOK'
      .format(str(AC_01),str(calc_ac_p0(omega_01, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A1*sinφ1 = {0} = {1}'.format(str(S_01),str(calc_as_p0(omega_01,[B_1, B_2], [Beta_1, Beta_2]))))
print('A2*cosφ2 = {0} = {1}'.format(str(AC_02),str(calc_ac_p0(omega_02, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A2*sinφ2 = {0} = {1}'.format(str(S_02),str(calc_as_p0(omega_02, [B_1, B_2], [Beta_1, Beta_2]))))

5.348575581961582 0.8010103924908942 2.2304647955848846e-05 0.000620977071733642 10 1.8768490993130438e-13 6.601731737231747e-11
βmの収束計算に10回。最後のβ1の残差が1.8768490993130438e-13。β2の残差が6.601731737231747e-11。
計算結果：h(t) = 0.212934526 + 5.348575581961582 * exp(-2.2304647955848846e-05 * t) + 0.8010103924908942 * exp(-0.000620977071733642 * t)
A1*cosφ1 = 6.0897718339999996 = 6.0897718339999996     ←左：入力値、右：単位応答からの計算値。一致してればOK
A1*sinφ1 = 0.515005366 = 0.5150053659999999
A2*cosφ2 = 5.1124602359999995 = 5.1124602289796455
A2*sinφ2 = 1.591950776 = 1.5919507738467806


In [48]:
""" 【計算例】単位温度励振時の貫流応答：計算がうまくいく例 """
""" セルの参照は「周期定常→指数項二項近似」シート内 """
#入力値
initBeta_1 = 10**(-10)                       #Betaの収束計算の初期値
B_00 = 0.212934526                           #定常項(熱貫流率)、セルR164C2
T_01 = 2                                     #一組目の周期[hour]、セルR194C3
omega_01 = 2 * cmath.pi / (T_01 * 3600)      #角周波数[rad/s]、セルR194C4
AC_01 = -0.211870646 + B_00                  #AcosφをC1(セルR194C7)と定常項から算出
S_01 = 0.000993134                           #Asinφ、セルR194C8
T_02 = 24                                    #二組目の周期[hour]、セルR194C9
omega_02 = 2 * cmath.pi / (T_02 * 3600)      #角周波数[rad/s]、セルR194C10
AC_02 = -0.220372378 + B_00                  #AcosφをC1(セルR194C13)と定常項から算出
S_02 = -0.060418385                          #Asinφ、セルR194C14

#Excelシート上の出力値
# B1 = -0.250692349                           セルR194C18
# B2 = 0.040236803                            セルR194C19
# Beta_1 = 2.39631E-05                        セルR194C16
# Beta_2 = 0.000177816                        セルR194C17

B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2 = calc_unitresponse(initBeta_1, B_00, omega_01, AC_01, S_01
                                                                          , omega_02, AC_02, S_02)
print (B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2)
print ('βmの収束計算に{0}回。最後のβ1の残差が{1}。β2の残差が{2}。'.format(abs(i),Diff_Beta_1,Diff_Beta_2))
PlusMinus00 = " + " if B_1 >= 0 else " - "
PlusMinus01 = " + " if B_2 >= 0 else " - "
OK_Not = "" if i >= 0 else "計算失敗 \n"
print ('計算結果：{}h(t) = {}{}{} * exp(-{} * t){}{} * exp(-{} * t)'
       .format(OK_Not,B_00,PlusMinus00,abs(B_1),Beta_1,PlusMinus01,abs(B_2),Beta_2))

""" 周波数応答への復号確認 """
print('A1*cosφ1 = {0} = {1}     ←左：入力値、右：単位応答からの計算値。一致してればOK'
      .format(str(AC_01),str(calc_ac_p0(omega_01, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A1*sinφ1 = {0} = {1}'.format(str(S_01),str(calc_as_p0(omega_01,[B_1, B_2], [Beta_1, Beta_2]))))
print('A2*cosφ2 = {0} = {1}'.format(str(AC_02),str(calc_ac_p0(omega_02, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A2*sinφ2 = {0} = {1}'.format(str(S_02),str(calc_as_p0(omega_02, [B_1, B_2], [Beta_1, Beta_2]))))

-0.25069233243776656 0.04023678658147573 2.3963113076264546e-05 0.00017781562880923757 17 3.4597430274136125e-12 8.012812050157969e-11
βmの収束計算に17回。最後のβ1の残差が3.4597430274136125e-12。β2の残差が8.012812050157969e-11。
計算結果：h(t) = 0.212934526 - 0.25069233243776656 * exp(-2.3963113076264546e-05 * t) + 0.04023678658147573 * exp(-0.00017781562880923757 * t)
A1*cosφ1 = 0.001063880000000017 = 0.0010638800000000517     ←左：入力値、右：単位応答からの計算値。一致してればOK
A1*sinφ1 = 0.000993134 = 0.0009931340000000023
A2*cosφ2 = -0.007437851999999995 = -0.007437845653458655
A2*sinφ2 = -0.060418385 = -0.060418382908710176


In [49]:
""" 【計算例】単位温度励振時の貫流応答：計算が失敗する例 """
""" セルの参照は「周期定常→指数項二項近似」シート内 """
#入力値
initBeta_1 = 10**(-10)                       #Betaの収束計算の初期値
B_00 = 0.212934526                           #定常項(熱貫流率)、セルR164C2
T_01 = 0.5                                   #一組目の周期[hour]、セルR172C3
omega_01 = 2 * cmath.pi / (T_01 * 3600)      #角周波数[rad/s]、セルR172C4
AC_01 = -0.212932071 + B_00                  #AcosφをC1(セルR172C7)と定常項から算出
S_01 = 4.74993E-06                           #Asinφ、セルR172C8
T_02 = 8                                     #二組目の周期[hour]、セルR172C9
omega_02 = 2 * cmath.pi / (T_02 * 3600)      #角周波数[rad/s]、セルR172C10
AC_02 = -0.229702311 + B_00                  #AcosφをC1(セルR172C13)と定常項から算出
S_02 = -0.008102008                          #Asinφ、セルR172C14

#Excelシート上の出力値
# B1 = -4.56144E+14                           セルR172C18
# B2 = 4.56144E+14                            セルR172C19
# Beta_1 = 5.79707E-05                        セルR172C16
# Beta_2 = 5.79707E-05                        セルR172C17

B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2 = calc_unitresponse(initBeta_1, B_00, omega_01, AC_01, S_01
                                                                          , omega_02, AC_02, S_02)
print (B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2)
print ('βmの収束計算に{0}回。最後のβ1の残差が{1}。β2の残差が{2}。'.format(abs(i),Diff_Beta_1,Diff_Beta_2))
PlusMinus00 = " + " if B_1 >= 0 else " - "
PlusMinus01 = " + " if B_2 >= 0 else " - "
OK_Not = "" if i >= 0 else "計算失敗 \n"
print ('計算結果：{}h(t) = {}{}{} * exp(-{} * t){}{} * exp(-{} * t)'
       .format(OK_Not,B_00,PlusMinus00,abs(B_1),Beta_1,PlusMinus01,abs(B_2),Beta_2))
   
""" 周波数応答への復号確認 """
print('A1*cosφ1 = {0} = {1}     ←左：入力値、右：単位応答からの計算値。一致してればOK'
      .format(str(AC_01),str(calc_ac_p0(omega_01, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A1*sinφ1 = {0} = {1}'.format(str(S_01),str(calc_as_p0(omega_01,[B_1, B_2], [Beta_1, Beta_2]))))
print('A2*cosφ2 = {0} = {1}'.format(str(AC_02),str(calc_ac_p0(omega_02, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A2*sinφ2 = {0} = {1}'.format(str(S_02),str(calc_as_p0(omega_02, [B_1, B_2], [Beta_1, Beta_2]))))

-281.896505578739 281.6836323969127 5.7950562179270013e-05 5.799443950423279e-05 -23 7.6987786896851e-09 9.054176625235093e-09
βmの収束計算に23回。最後のβ1の残差が7.6987786896851e-09。β2の残差が9.054176625235093e-09。
計算結果：計算失敗 
h(t) = 0.212934526 - 281.896505578739 * exp(-5.7950562179270013e-05 * t) + 281.6836323969127 * exp(-5.799443950423279e-05 * t)
A1*cosφ1 = 2.4550000000123084e-06 = 2.4550000716772047e-06     ←左：入力値、右：単位応答からの計算値。一致してればOK
A1*sinφ1 = 4.74993e-06 = 4.7499299995479305e-06
A2*cosφ2 = -0.01676778499999998 = -0.01217618490068162
A2*sinφ2 = -0.008102008 = -0.0068823605013932365


In [50]:
""" 【計算例】表面熱流励振時の励振側温度応答 """
""" セルの参照は「周期定常→指数項二項近似」シート内 """
#入力値
initBeta_1 = 10**(-10)                       #Betaの収束計算の初期値
B_00 = 4.547025536                           #定常項(室内側表面熱伝達率を含まない熱貫流率の逆数)、セルR244C2
T_01 = 2                                     #一組目の周期[hour]、セルR274C3
omega_01 = 2 * cmath.pi / (T_01 * 3600)      #角周波数[rad/s]、セルR274C4
AC_01 = -4.533235575 + B_00                  #AcosφをC1(セルR274C7)と定常項から算出
S_01 = -0.013788427                          #Asinφ、セルR172C8
T_02 = 24                                    #二組目の周期[hour]、セルR274C9
omega_02 = 2 * cmath.pi / (T_02 * 3600)      #角周波数[rad/s]、セルR274C10
AC_02 = -4.517968049 + B_00                  #AcosφをC1(セルR274C13)と定常項から算出
S_02 = -0.055523695                          #Asinφ、セルR274C14

#Excelシート上の出力値
# B1 = -4.518220113                           セルR274C18
# B2 = -0.020888737                           セルR274C19
# Beta_1 = 8.49748E-07                        セルR274C16
# Beta_2 = 0.000545504                        セルR274C17

B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2 = calc_unitresponse(initBeta_1, B_00, omega_01, AC_01, S_01
                                                                          , omega_02, AC_02, S_02)
print (B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2)
print ('βmの収束計算に{0}回。最後のβ1の残差が{1}。β2の残差が{2}。'.format(abs(i),Diff_Beta_1,Diff_Beta_2))
PlusMinus00 = " + " if B_1 >= 0 else " - "
PlusMinus01 = " + " if B_2 >= 0 else " - "
OK_Not = "" if i >= 0 else "計算失敗 \n"
print ('計算結果：{}h(t) = {}{}{} * exp(-{} * t){}{} * exp(-{} * t)'
       .format(OK_Not,B_00,PlusMinus00,abs(B_1),Beta_1,PlusMinus01,abs(B_2),Beta_2))

""" 周波数応答への復号確認 """
print('A1*cosφ1 = {0} = {1}     ←左：入力値、右：単位応答からの計算値。一致してればOK'
      .format(str(AC_01),str(calc_ac_p0(omega_01, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A1*sinφ1 = {0} = {1}'.format(str(S_01),str(calc_as_p0(omega_01,[B_1, B_2], [Beta_1, Beta_2]))))
print('A2*cosφ2 = {0} = {1}'.format(str(AC_02),str(calc_ac_p0(omega_02, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A2*sinφ2 = {0} = {1}'.format(str(S_02),str(calc_as_p0(omega_02, [B_1, B_2], [Beta_1, Beta_2]))))

-4.518220112768168 -0.020888737129292113 8.497477267530519e-07 0.0005455038134543818 5 6.40705673940283e-16 5.5266692914821e-12
βmの収束計算に5回。最後のβ1の残差が6.40705673940283e-16。β2の残差が5.5266692914821e-12。
計算結果：h(t) = 4.547025536 - 4.518220112768168 * exp(-8.497477267530519e-07 * t) - 0.020888737129292113 * exp(-0.0005455038134543818 * t)
A1*cosφ1 = 0.013789960999999629 = 0.013789961000000395     ←左：入力値、右：単位応答からの計算値。一致してればOK
A1*sinφ1 = -0.013788427 = -0.013788427
A2*cosφ2 = 0.029057486999999327 = 0.029057487000923192
A2*sinφ2 = -0.055523695 = -0.05552369499998921


In [51]:
""" 【計算例】表面熱流励振時の貫流側熱流応答 """
""" セルの参照は「周期定常→指数項二項近似」シート内 """
#入力値
initBeta_1 = 10**(-10)                       #Betaの収束計算の初期値
B_00 = 1                                     #定常項(熱流の収束値)、セルR324C2
T_01 = 2                                     #一組目の周期[hour]、セルR354C3
omega_01 = 2 * cmath.pi / (T_01 * 3600)      #角周波数[rad/s]、セルR354C4
AC_01 = -0.999812847 + B_00                  #AcosφをC1(セルR354C7)と定常項から算出
S_01 = 0.000147255                           #Asinφ、セルR354C8
T_02 = 24                                    #二組目の周期[hour]、セルR354C9
omega_02 = 2 * cmath.pi / (T_02 * 3600)      #角周波数[rad/s]、セルR354C10
AC_02 = -1.004680904 + B_00                  #AcosφをC1(セルR354C13)と定常項から算出
S_02 = -0.010360299                          #Asinφ、セルR354C14

#Excelシート上の出力値
# B1 = -1.005740046                           セルR354C18
# B2 = 0.006166047                            セルR354C19
# Beta_1 = 9.06877E-07                        セルR354C16
# Beta_2 = 0.000175593                        セルR354C17

B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2 = calc_unitresponse(initBeta_1, B_00, omega_01, AC_01, S_01
                                                                          , omega_02, AC_02, S_02)
print (B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2)
print ('βmの収束計算に{0}回。最後のβ1の残差が{1}。β2の残差が{2}。'.format(abs(i),Diff_Beta_1,Diff_Beta_2))
PlusMinus00 = " + " if B_1 >= 0 else " - "
PlusMinus01 = " + " if B_2 >= 0 else " - "
OK_Not = "" if i >= 0 else "計算失敗 \n"
print ('計算結果：{}h(t) = {}{}{} * exp(-{} * t){}{} * exp(-{} * t)'
       .format(OK_Not,B_00,PlusMinus00,abs(B_1),Beta_1,PlusMinus01,abs(B_2),Beta_2))

""" 周波数応答への復号確認 """
print('A1*cosφ1 = {0} = {1}     ←左：入力値、右：単位応答からの計算値。一致してればOK'
      .format(str(AC_01),str(calc_ac_p0(omega_01, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A1*sinφ1 = {0} = {1}'.format(str(S_01),str(calc_as_p0(omega_01,[B_1, B_2], [Beta_1, Beta_2]))))
print('A2*cosφ2 = {0} = {1}'.format(str(AC_02),str(calc_ac_p0(omega_02, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A2*sinφ2 = {0} = {1}'.format(str(S_02),str(calc_as_p0(omega_02, [B_1, B_2], [Beta_1, Beta_2]))))

-1.0057400458746013 0.006166046532845512 9.068773967917873e-07 0.00017559337909684678 5 7.603738111020128e-14 4.429107910508892e-11
βmの収束計算に5回。最後のβ1の残差が7.603738111020128e-14。β2の残差が4.429107910508892e-11。
計算結果：h(t) = 1 - 1.0057400458746013 * exp(-9.068773967917873e-07 * t) + 0.006166046532845512 * exp(-0.00017559337909684678 * t)
A1*cosφ1 = 0.00018715300000005097 = 0.00018715299999998505     ←左：入力値、右：単位応答からの計算値。一致してればOK
A1*sinφ1 = 0.000147255 = 0.000147255
A2*cosφ2 = -0.004680903999999986 = -0.004680903973962904
A2*sinφ2 = -0.010360299 = -0.010360298999675305


In [52]:
""" 【計算例】表面熱流励振時の貫流側熱流応答：計算が失敗する例 """
""" セルの参照は「周期定常→指数項二項近似」シート内 """
#入力値
initBeta_1 = 10**(-10)                       #Betaの収束計算の初期値
B_00 = 1                                     #定常項(熱流の収束値)、セルR324C2
T_01 = 1                                     #一組目の周期[hour]、セルR338C3
omega_01 = 2 * cmath.pi / (T_01 * 3600)      #角周波数[rad/s]、セルR338C4
AC_01 = -1.000004848 + B_00                  #AcosφをC1(セルR338C7)と定常項から算出
S_01 = -2.32365E-05                          #Asinφ、セルR338C8
T_02 = 2                                     #二組目の周期[hour]、セルR338C9
omega_02 = 2 * cmath.pi / (T_02 * 3600)      #角周波数[rad/s]、セルR338C10
AC_02 = -0.999812847 + B_00                  #AcosφをC1(セルR338C13)と定常項から算出
S_02 = 0.000147255                           #Asinφ、セルR338C14

#Excelシート上の出力値
# B1 = #N/A                                   セルR338C18
# B2 = #N/A                                   セルR338C19
# Beta_1 = -4.06099E-07                       セルR338C16
# Beta_2 = 0.000881259                        セルR338C17

B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2 = calc_unitresponse(initBeta_1, B_00, omega_01, AC_01, S_01
                                                                          , omega_02, AC_02, S_02)
print (B_1, B_2, Beta_1, Beta_2, i, Diff_Beta_1, Diff_Beta_2)
print ('βmの収束計算に{0}回。最後のβ1の残差が{1}。β2の残差が{2}。'.format(abs(i),Diff_Beta_1,Diff_Beta_2))
PlusMinus00 = " + " if B_1 >= 0 else " - "
PlusMinus01 = " + " if B_2 >= 0 else " - "
OK_Not = "" if i >= 0 else "計算失敗 \n"
print ('計算結果：{}h(t) = {}{}{} * exp(-{} * t){}{} * exp(-{} * t)'
       .format(OK_Not,B_00,PlusMinus00,abs(B_1),Beta_1,PlusMinus01,abs(B_2),Beta_2))

""" 周波数応答への復号確認 """
print('A1*cosφ1 = {0} = {1}     ←左：入力値、右：単位応答からの計算値。一致してればOK'
      .format(str(AC_01),str(calc_ac_p0(omega_01, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A1*sinφ1 = {0} = {1}'.format(str(S_01),str(calc_as_p0(omega_01,[B_1, B_2], [Beta_1, Beta_2]))))
print('A2*cosφ2 = {0} = {1}'.format(str(AC_02),str(calc_ac_p0(omega_02, B_00, [B_1, B_2], [Beta_1, Beta_2]))))
print('A2*sinφ2 = {0} = {1}'.format(str(S_02),str(calc_as_p0(omega_02, [B_1, B_2], [Beta_1, Beta_2]))))

-0.99949829729322 -0.0006357630716612175 -4.0609870730611923e-07 0.0008812594579881175 -3 6.930469067515491e-13 1.3802360798341115e-09
βmの収束計算に3回。最後のβ1の残差が6.930469067515491e-13。β2の残差が1.3802360798341115e-09。
計算結果：計算失敗 
h(t) = 1 - 0.99949829729322 * exp(--4.0609870730611923e-07 * t) - 0.0006357630716612175 * exp(-0.0008812594579881175 * t)
A1*cosφ1 = -4.8479999998907175e-06 = -4.8479999999015595e-06     ←左：入力値、右：単位応答からの計算値。一致してればOK
A1*sinφ1 = -2.32365e-05 = -2.3236499999999973e-05
A2*cosφ2 = 0.00018715300000005097 = 0.00018715300055420774
A2*sinφ2 = 0.000147255 = 0.00014725499999974212


#### (2) 単位応答から復号した周波数応答と精算値の誤差の確認 

In [53]:
""" 二つの三角関数間の誤差算定モジュール `calc_rmse` """ 

#入力値(その1)
B_0 = 0.212934526                           #セルR79C27
B_1 = 5.348575591                           #セルR114C18
B_2 = 0.801010388                           #セルR114C19
Beta_1 = 2.23046E-05                        #セルR114C16
Beta_2 = 0.000620977                        #セルR114C17

T = 2                                       #RMSEを求める周期[hour]、セルR80C30
omega = 2 * cmath.pi / (T * 3600)           #角周波数[rad/s]、セルR81C30
AC_2 = 6.089771834     #上記TにおけるAcosφの精算値, セルR84C30
AS_2 = 0.515005366     #上記TにおけるAsinφの精算値, セルR85C30
#Excelシート上の出力値
# RMSE = 0                                 #セルR114C30
AC_1 = calc_ac_p0(omega, B_0, [B_1, B_2], [Beta_1, Beta_2])
AS_1 = calc_as_p0(omega, [B_1, B_2], [Beta_1, Beta_2])
RMSE=calc_rmse(AC_1, AS_1, AC_2, AS_2)
print('T = {} : RMSE = {}'.format(T,RMSE))

#入力値(その2)
T = 8                                       #RMSEを求める周期[hour]、セルR80C34
omega = 2 * cmath.pi / (T * 3600)           #角周波数[rad/s]、セルR81C34
AC_2 = 5.610566878     #上記TにおけるAcosφの精算値, セルR84C34
AS_2 = 0.812802134     #上記TにおけるAsinφの精算値, セルR85C34
#Excelシート上の出力値
# RMSE = 0.01890851                         #セルR114C34
AC_1 = calc_ac_p0(omega, B_0, [B_1, B_2], [Beta_1, Beta_2])
AS_1 = calc_as_p0(omega, [B_1, B_2], [Beta_1, Beta_2])
RMSE=calc_rmse(AC_1, AS_1, AC_2, AS_2)
print('T = {} : RMSE = {}'.format(T,RMSE))

#入力値(その3)
T = 24                                       #RMSEを求める周期[hour]、セルR80C37
omega = 2 * cmath.pi / (T * 3600)           #角周波数[rad/s]、セルR81C37
AC_2 = 5.112460236     #上記TにおけるAcosφの精算値, セルR84C37
AS_2 = 1.591950776     #上記TにおけるAsinφの精算値, セルR85C37
#Excelシート上の出力値
# RMSE = 6.47366E-16                         #セルR114C37
AC_1 = calc_ac_p0(omega, B_0, [B_1, B_2], [Beta_1, Beta_2])
AS_1 = calc_as_p0(omega, [B_1, B_2], [Beta_1, Beta_2])
RMSE=calc_rmse(AC_1, AS_1, AC_2, AS_2)
print('T = {} : RMSE = {}'.format(T,RMSE))

#入力値(その4)
T = 48                                       #RMSEを求める周期[hour]、セルR80C39
omega = 2 * cmath.pi / (T * 3600)           #角周波数[rad/s]、セルR81C39
AC_2 = 4.113083518     #上記TにおけるAcosφの精算値, セルR84C39
AS_2 = 2.422580775     #上記TにおけるAsinφの精算値, セルR85C39
#Excelシート上の出力値
# RMSE = 0.009744509                         #セルR114C39
AC_1 = calc_ac_p0(omega, B_0, [B_1, B_2], [Beta_1, Beta_2])
AS_1 = calc_as_p0(omega, [B_1, B_2], [Beta_1, Beta_2])
RMSE=calc_rmse(AC_1, AS_1, AC_2, AS_2)
print('T = {} : RMSE = {}'.format(T,RMSE))

T = 2 : RMSE = 2.232893942265197e-07
T = 8 : RMSE = 0.01890902128330531
T = 24 : RMSE = 2.275016978442871e-06
T = 48 : RMSE = 0.009740928427629296


### 3.5 計算式の導出過程
- 西澤、三浦：周波数特性から単位応答を近似的に求める方法に関する検討, 日本建築学会大会学術講演梗概集D2, pp.586-587, 2017
- 以下、「数字の」式番号は上記文献に対応

#### (1) 周波数応答と単位応答の係数の関係式の導出

時刻$\;t = 0\;[sec]$での$1K$励振に対する単位応答$\;h(t)$$[W/m ^ 2K]\;$を、

$$ \\ \qquad h(t) = B_0 + \sum_m B_m e ^ {- \beta_m t} \qquad (1) \\ $$

とし、$\theta = \cos \omega_p t \;[K] \;(\omega_p$：角周波数$[rad/s]$, $\omega_p = 2\pi/T/3600$, $T$：周期$[hour])$の変動に対して、

- ISO 13786, Thermal performance of building components - Dynamic thermal　characteristics - Calculation methods, 2007
- 建築学便覧 Ⅰ計画, 丸善, 1980

等で求まる周波数応答$H_p(t)[W/m ^ 2K]$を、

$$H_p(t) = A_p \cos (\omega_p t + \varphi_p) = A_p \cos \varphi_p \cos \omega_p t - A_p \sin \varphi_p \sin \omega_p t \qquad (2) \\ $$

とすると、$H_p(t)$は、$h(t)$の畳み込み積分で、

$$ \begin{align}
H_p(t) &= \int_0^t \theta'(\tau)\,h(t-\tau)\;d\tau + \theta(0)\,h(t) \\
&= \int_0^t (-\omega_p \sin \omega_p \tau) \Bigl\{ B_0 + \sum_m B_m e ^ {- \beta_m (t - \tau)} \Bigr\} d\tau  + \theta(0)\,h(t) \\
&\qquad 
\begin{pmatrix}
\int_0^t (-\omega_p \sin \omega_p \tau) B_0 d\tau = B_0 \bigl[ \cos \omega_p \tau \bigr]_0^t = B_0 (\cos \omega_p t - 1) \\
\\
\int_0^t (-\omega_p \sin \omega_p \tau) B_m e ^ {- \beta_m (t - \tau)} d\tau = - B_m \omega_p e ^ {- \beta_m t} \int_0^t e ^ {\beta_m \tau} \sin \omega_p \tau d\tau \\
= - B_m \omega_p e ^ {- \beta_m t} \bigl[ \beta_m e ^ {\beta_m \tau} \sin \omega_p \tau - \omega_p e ^ {\beta_m \tau} \cos \omega_p \tau  \bigr]_0^t \; \big/ \; ({\beta_m}^2 + {\omega_p}^2) \\
= (B_m {\omega_p}^2 \cos \omega_p t - B_m \beta_m \omega_p \sin \omega_p t - B_m {\omega_p}^2 e ^ {- \beta_m t})\; \big/ \; ({\beta_m}^2 + {\omega_p}^2) \\
\end{pmatrix}
\\
&= B_0 (\cos \omega_p t - 1) + \sum_m \biggl( \frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2} \cos \omega_p t - \frac{B_m {\beta_m}{\omega_p}}{{\beta_m}^2 + {\omega_p}^2} \sin \omega_p t - \frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2} e ^ {- \beta_m t} \biggr) + B_0 + \sum_m B_m e ^ {- \beta_m t}
\end{align} $$

周期定常を考えると$\;t \rightarrow \infty \;$で$\;e ^ {- \beta_m t}\rightarrow 0\;$となることから、$H_p(t)$は、

$$ H_p(t) = \biggl( B_0 + \sum_m \frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2} \biggr)\cos \omega_p t
- \sum_m \frac{B_m \beta_m \omega_p}{{\beta_m}^2 + {\omega_p}^2} \sin \omega_p t \qquad (3) $$

と求まる。式$(2)$、$(3)$から、

$$ A_p \cos \varphi_p = B_0 + \sum_m \frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2} \qquad (4') $$
$$ A_p \sin \varphi_p = \sum_m \frac{B_m \beta_m \omega_p}{{\beta_m}^2 + {\omega_p}^2} \qquad (5') $$

となり、周波数応答と単位応答の係数の関係式が得られる。

#### (2) 周波数応答からの指数項2項の単位応答での近似

式$(3)$は、無限項からなる単位応答で成り立つが、指数項が数項の単位応答であっても式$(3)$を満たすのであれば、その単位応答は、その一次元躯体構成の単位応答の近似解として扱うことができる。

指数項1項で近似することを考えると、式$(4')$、$(5')$は容易に解くことができるが、指数項1項での近似は精度を確保することが難しいことが確認されている。

一方、指数項3項で近似することを考えると、式$(4')$、$(5')$を解くことで得られる$\beta_m$の式(式$(8)$に対応し、3つの式の連立式となる)の解を得ることが難しい状況が現れる(特に貫流応答において)ことが確認されている。

指数項4項以上を考える際には、そもそも式$(8)$に対応する式の導出事態が困難を極める。

指数項2項での近似では、比較的簡単に$\beta_m$を収束計算により求めることが可能であったことから、指数項2項での近似を検討することとした。

指数項2項での近似を考えると、式$(4')$、$(5')$に変数$C_p$、$S_p$を導入した式

$$ A_p \cos \varphi_p = B_0 + \sum_{m=1}^M \frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2} = B_0 + C_p \qquad (4) $$
$$ A_p \sin \varphi_p = \sum_{m=1}^M \frac{B_m \beta_m \omega_p}{{\beta_m}^2 + {\omega_p}^2} = S_p  \qquad (5) $$

において、$M=2$として、二組の$\omega_p$、$C_p$、$S_p$を用いて算出することができる。$p=1, \; q=2, \; m=1, \; n=2$として書き下すと、

$$C_p=\frac{B_m {\omega_p}^2}{{\beta_m}^2 + {\omega_p}^2}+ \frac{B_n {\omega_p}^2}{{\beta_n}^2 + {\omega_p}^2}, \quad S_p = \frac{B_m \beta_m \omega_p}{{\beta_m}^2 + {\omega_p}^2} + \frac{B_n \beta_n \omega_p}{{\beta_n}^2 + {\omega_p}^2} \qquad (a),\; (b)$$

$$C_q=\frac{B_m {\omega_q}^2}{{\beta_m}^2 + {\omega_q}^2}+ \frac{B_n {\omega_q}^2}{{\beta_n}^2 + {\omega_q}^2}, \quad S_q = \frac{B_m \beta_m \omega_q}{{\beta_m}^2 + {\omega_q}^2} + \frac{B_n \beta_n \omega_q}{{\beta_n}^2 + {\omega_q}^2} \qquad (c),\; (d)$$

$$(a) \times \beta_m - (b) \times \omega_p \; \Rightarrow \; B_n = \frac{{\beta_n}^2 + {\omega_p}^2}{{\omega_p}^2(\beta_m-\beta_n)} (\beta_m C_p - \omega_p S_p) \qquad (e)$$

$$(a) \times \beta_n - (b) \times \omega_p \; \Rightarrow \; B_m = \frac{{\beta_m}^2 + {\omega_p}^2}{{\omega_p}^2(\beta_n-\beta_m)} (\beta_n C_p - \omega_p S_p) \qquad (f)$$

$$(c) \times \beta_m - (d) \times \omega_q \; \Rightarrow \; B_n = \frac{{\beta_n}^2 + {\omega_q}^2}{{\omega_q}^2(\beta_m-\beta_n)} (\beta_m C_q - \omega_q S_q) \qquad (g)$$

$$(c) \times \beta_n - (d) \times \omega_q \; \Rightarrow \; B_m = \frac{{\beta_m}^2 + {\omega_q}^2}{{\omega_q}^2(\beta_n-\beta_m)} (\beta_n C_q - \omega_q S_q) \qquad (h)$$

$$ \begin{align}
(e)=(g) \; &\Rightarrow \; \frac{{\beta_n}^2 + {\omega_p}^2}{{\omega_p}^2(\beta_m-\beta_n)} (\beta_m C_p - \omega_p S_p) = \frac{{\beta_n}^2 + {\omega_q}^2}{{\omega_q}^2(\beta_m-\beta_n)} (\beta_m C_q - \omega_q S_q) \\\\
\; &\Rightarrow \;{\omega_q}^2 ({\beta_n}^2 + {\omega_p}^2)(\beta_m C_p - \omega_p S_p) = {\omega_p}^2 ({\beta_n}^2 + {\omega_q}^2)  (\beta_m C_q - \omega_q S_q) \\\\
\; &\Rightarrow \; \beta_m = \frac{\omega_p {\omega_q}^2 S_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 \omega_q S_q ( {\beta_n}^2 + {\omega_q}^2 )}{{\omega_q}^2 C_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 C_q ( {\beta_n}^2 + {\omega_q}^2 )} \qquad (i)
\end{align} $$

$$ \begin{align}
(f)=(h) \; &\Rightarrow \; \frac{{\beta_m}^2 + {\omega_p}^2}{{\omega_p}^2(\beta_n-\beta_m)} (\beta_n C_p - \omega_p S_p) = \frac{{\beta_m}^2 + {\omega_q}^2}{{\omega_q}^2(\beta_n-\beta_m)} (\beta_n C_q - \omega_q S_q) \\\\
\; &\Rightarrow \; {\omega_q}^2 ({\beta_m}^2 + {\omega_p}^2)(\beta_n C_p - \omega_p S_p) = {\omega_p}^2 ({\beta_m}^2 + {\omega_q}^2)  (\beta_n C_q - \omega_q S_q) \\\\
\; &\Rightarrow \; \beta_n = \frac{\omega_p {\omega_q}^2 S_p ( {\beta_m}^2 + {\omega_p}^2 ) - {\omega_p}^2 \omega_q S_q ( {\beta_m}^2 + {\omega_q}^2 )}{{\omega_q}^2 C_p ( {\beta_m}^2 + {\omega_p}^2 ) - {\omega_p}^2 C_q ( {\beta_m}^2 + {\omega_q}^2 )} \qquad (j)
\end{align} $$

式$(i)$、$(j)$の$m$、$n$をまとめると式$(8)$を、式$(e)$～$(h)$をまとめると式$(9)$となる。

$$ \qquad 
\beta_m
= \frac{\omega_p {\omega_q}^2 S_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 \omega_q S_q ( {\beta_n}^2 + {\omega_q}^2 )}
{{\omega_q}^2 C_p ( {\beta_n}^2 + {\omega_p}^2 ) - {\omega_p}^2 C_q ( {\beta_n}^2 + {\omega_q}^2 )}
, \quad
( m = 1,\! 2,\; n = 3 - m, \; p = 1, \; q = 2)
\qquad (8)
\\
$$
$$ \qquad 
B_m
= \frac{{\beta_m}^2 + {\omega_p}^2}{{\omega_p}^2 ( \beta_n - \beta_m )} ( \beta_n C_p - \omega_p S_p )
, \quad
( m = 1,\! 2,\; n = 3 - m, \; p = 1,\! 2)
\qquad (9)
\\
$$

ただし、$\beta_m$は単位応答の時定数であることから、$\beta_m>0$となる必要がある。また、指数項が2項となるために、$\beta_1 \neq \beta_2$となる必要がある。


以上の導出は、温度励振における熱流の単位応答と周波数応答を念頭において行っているが、温度励振側の吸熱応答も温度固定側の貫流応答も同様に扱うことが可能である。また、表面熱流励振、裏面温度固定の条件における表面温度応答、裏面熱流応答においても本質的に変わりはなく、単位応答を式$(1)$、周波数応答を式$(2)$で表せる以上は、二組の周波数応答から式$(8)$、式$(9)$で単位応答の近似解を求めることができる。

#### (3) 二つの三角関数間の誤差

同じ角周波数$\omega$の二つの三角関数$ \; A_{c1} \cos \omega t - A_{s1} \sin \omega t \; $と$ \; A_{c2} \cos \omega t - A_{s2} \sin \omega t \; $ の平均平方二乗誤差$RMSE$を下式で求める。

$$ \begin{align}
RMSE &= \sqrt{ \frac{1}{T} \int_0^T \{ (A_{c1} \cos \omega t - A_{s1} \sin \omega t) - (A_{c2} \cos \omega t - A_{s2} \sin \omega t) \}^2 dt }\\
&= \sqrt{ \frac{1}{T} \int_0^T \{ (A_{c1} - A_{c2}) \cos \omega t - (A_{s1} - A_{s2}) \sin \omega t \}^2 dt }\\
&= \sqrt{ \frac{ (A_{c1} - A_{c2})^2 + (A_{s1} - A_{s2})^2 }{T} \int_0^T \cos^2 \Bigl( \omega t + \tan^{-1} \frac{A_{s1} - A_{s2}}{A_{c1} - A_{c2}} \Bigr)  dt } \\
&= \sqrt{ \frac{ (A_{c1} - A_{c2})^2 + (A_{s1} - A_{s2})^2 }{2 T} \int_0^T \Bigl\{ \cos \Bigl( 2 \omega t + 2 \tan^{-1} \frac{A_{s1} - A_{s2}}{A_{c1} - A_{c2}} \Bigr) +1 \Bigr\} dt } \\
&= \sqrt{ \frac{ (A_{c1} - A_{c2})^2 + (A_{s1} - A_{s2})^2 }{2 T} }\sqrt{ \Bigl[ \frac{1}{2 \omega} \sin \Bigl( 2 \omega t + 2 \tan^{-1} \frac{A_{s1} - A_{s2}}{A_{c1} - A_{c2}} \Bigr) + t \Bigr]_0^T} \\
&= \sqrt{ \frac{ (A_{c1} - A_{c2})^2 + (A_{s1} - A_{s2})^2 }{2} } \qquad (k)\\
\end{align} $$


## 4. 複数の単位応答から有効熱容量を算出するルート

### 4.1 入力値と出力値

#### (1) 入力値




#### (2) 引数と入力値


   

#### (3) 出力値





#### (4) 戻り値と出力値


    


### 4.2 計算の流れ


#### (1) 単位応答から周波数応答への変換

- 単位応答から角周波数$\omega$における周波数応答に変換する式

$$ 
A \cos \varphi =  B_0 +  \sum_{m=1}^M \frac{B_m {\omega}^2}{{\beta_m}^2 + {\omega}^2} \quad (1) \\
A \sin \varphi = \sum_{m=1}^M \frac{B_m \beta_m \omega}{{\beta_m}^2 + {\omega}^2}  \quad (2) \\
$$

#### (2) 「表面熱伝達層を含まない」応答から「表面熱伝達層を含む」応答への変換

### 4.3 計算プログラム

### 4.4 計算例


### 4.5 計算式の導出過程

#### (1) 「表面熱伝達層を含まない」応答から「表面熱伝達層を含む」応答への変換

「表面熱伝達層を含まない」躯体構成において、$a$側$(n=2)$を熱流の単位振幅励振、$b$側$(n=N)$を温度固定とした場合($\hat{q}_a = \cos \omega t$、$\hat{\theta}_b = 0\:$)の$a$側表面温度応答$\;\hat{\theta}_a\;$と、$b$側$(n=N)$を温度の単位振幅励振、$a$側$(n=2)$を熱流固定とした場合($\hat{\theta}_b = \cos \omega t$、$\hat{q}_a = 0\:$)の$a$側表面温度応答$\;\hat{\theta}_{ar}\;$が下式

$$
\left\{
\begin{array}{ll}
\hat{\theta}_a = \frac{\sqrt{{{R_{12}}'}^2 + {{I_{12}}'}^2}}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \;\cos \Bigl( \omega t + \tan^{-1} \frac{{I_{12}}'}{{R_{12}}'} - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'}+ \pi \Bigr) = {A_a}' \cos {\varphi_a}' \cos \omega t - {A_a}' \sin {\varphi_a}' \sin \omega t 
\\
\hat{\theta}_{ar} = \frac{1}{\sqrt{{{R_{11}}'}^2 + {{I_{11}}'}^2}} \; \cos \Bigl( \omega t - \tan^{-1} \frac{{I_{11}}'}{{R_{11}}'} \Bigr) = {A_b}' \cos {\varphi_b}' \cos \omega t - {A_b}' \sin {\varphi_b}' \sin \omega t 
\end{array}
\right.  \\
\hspace{250pt} \qquad (1) \; (\leftarrow 1.2 \; 式(19))
$$

で得られているときに、層$1$$(n=1)$として表面熱伝達層を加えて「表面熱伝達層を含む」構成とした場合における、温度励振時の熱流応答を導出する。

式$(1)$で、係数${A_a}' \cos {\varphi_a}'$, ${A_a}' \sin {\varphi_a}'$, ${A_b}' \cos {\varphi_b}'$, ${A_b}' \sin {\varphi_b}'$が与えられている時に、式$(1)$第$2$式から、

$$
{R_{11}}'= \frac{ {A_b}' \cos {\varphi_b}' }{ ( {A_b}' \cos {\varphi_b}' )^2 + ( {A_b}' \sin {\varphi_b}' )^2 } = \frac{ \cos {\varphi_b}' }{ {A_b}' } \qquad (2) \;
$$

$$
{I_{11}}'= - \frac{ {A_b}' \sin {\varphi_b}' }{ ( {A_b}' \cos {\varphi_b}' )^2 + ( {A_b}' \sin {\varphi_b}' )^2 } = - \frac{ \sin {\varphi_b}' }{ {A_b}' } \qquad (3) \;
$$

が得られる。また、式$(1)$第$1$式から、

$$
{R_{12}}'= \frac{ \cos ({\varphi_a}' - {\varphi_b}' - \pi) }{ {A_a}' } 
= \frac{ -\cos {\varphi_a}' \cos {\varphi_b}' - \sin {\varphi_a}' \sin {\varphi_b}' }{ {A_a}' }
\qquad (4) \;
$$

$$
{I_{12}}'= \frac{ \sin ({\varphi_a}' - {\varphi_b}' - \pi) }{ {A_a}' } 
= \frac{ -\sin {\varphi_a}' \cos {\varphi_b}' + \cos {\varphi_a}' \sin {\varphi_b}' }{ {A_a}' }
\qquad (5) \;
$$

が得られる。


式$(2)$～$(5)$で得られる$\;{R_{11}}'$、${I_{11}}'$、${R_{12}}'$、${I_{12}}'\;$は、層$2$～$N$の四端子行列

$$\textbf{Z}_{2N} = \begin{pmatrix} {Z_{11}}' & {Z_{12}}' \\ {Z_{21}}' & {Z_{22}}' \end{pmatrix} = \begin{pmatrix} {R_{11}}'+j{I_{11}}' & {R_{12}}'+j{I_{12}}' \\ {R_{21}}'+j{I_{21}}' & {R_{22}}'+j{I_{22}}' \end{pmatrix}=\textbf{Z}_{N} \; \textbf{Z}_{N-1} \;  \cdots \;  \textbf{Z}_{2} \qquad (6) \; (\leftarrow 1.2 \; 式(16))$$

の要素であり、層$1$の表面熱伝達層$(n=1$, 表面熱伝達率$\alpha_s)$ の行列

$$ \;\textbf{Z}_{1} = \begin{pmatrix} 1 & -1/\alpha_s \\ 0 & 1 \end{pmatrix} \qquad (7) \; (\leftarrow 1.2 式(14)) $$

をかけて層$1$～$N$の四端子行列$\textbf{Z}$を求めると、

$$\begin{align}
\textbf{Z} &= \textbf{Z}_{2N} \; \textbf{Z}_{1} 
= \begin{pmatrix} {Z_{11}}' & {Z_{12}}' \\ {Z_{21}}' & {Z_{22}}' \end{pmatrix}
  \begin{pmatrix} 1 & -1/\alpha_s \\ 0 & 1 \end{pmatrix} \\
&= \begin{pmatrix} {Z_{11}}' & - {Z_{11}}' \big/ \alpha_s + {Z_{12}}' \\ {Z_{21}}' & - {Z_{21}}' \big/ \alpha_s + {Z_{22}}' \end{pmatrix} = \begin{pmatrix} Z_{11} & Z_{12} \\ Z_{21} & Z_{22} \end{pmatrix}
\qquad (8) 
\end{align}
$$

となり、四端子行列$\textbf{Z}$の要素$R_{11}$、$I_{11}$、$R_{12}$、$I_{12}$は、

$$
\left\{
\begin{array}{ll}
R_{11} = {R_{11}}' \\
I_{11} = {I_{11}}' \\
R_{12} = - {R_{11}}' \big/ \alpha_s + {R_{12}}' \\
I_{12} = - {I_{11}}' \big/ \alpha_s + {I_{12}}'
\end{array}
\right.  \qquad (9) \
$$

と求まる。


「表面熱伝達層を含む」躯体構成において、$a$側$(n=1)$で単位温度振幅で励振する場合($\hat{\theta}_a = \cos \omega t$、$\hat{\theta}_b = 0$)の$a$側熱流応答$\;\hat{q}_a\;$と、$b$側$(n=N)$で単位温度振幅で励振する場合($\hat{\theta}_b = \cos \omega t$、$\hat{\theta}_a = 0$)の$a$側熱流応答$\;\hat{q}_b\;$ は、$\textbf{Z}$の要素から下式

$$
\left\{
\begin{array}{ll}
\hat{q}_a = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\cos \Bigl( \omega t +  \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi \Bigr) = {A_a} \cos {\varphi_a} \cos \omega t - {A_a} \sin {\varphi_a} \sin \omega t 
\\
\hat{q}_b = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \; \cos \Bigl( \omega t - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi \Bigr) = {A_b} \cos {\varphi_b} \cos \omega t - {A_b} \sin {\varphi_b} \sin \omega t 
\end{array}
\right.  \qquad (10) \; (\leftarrow 1.2 式(17))
$$

で求めることができることから、「表面熱伝達層を含む」躯体構成における熱流応答の係数は、式$(9)$を用いて


$$
\left\{
\begin{array}{ll}
A_a \cos \varphi_a = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\cos \Bigl( \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi \Bigr) \\
A_a \sin \varphi_a = \frac{\sqrt{{R_{11}}^2 + {I_{11}}^2}}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \;\sin \Bigl(   \tan^{-1} \frac{I_{11}}{R_{11}} - \tan^{-1} \frac{I_{12}}{R_{12}} + \pi \Bigr) \\
A_b \cos \varphi_b = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \; \cos \Bigl( - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi \Bigr) \\
A_b \sin \varphi_b = \frac{1}{\sqrt{{R_{12}}^2 + {I_{12}}^2}} \; \sin \Bigl( - \tan^{-1} \frac{I_{12}}{R_{12}} - \pi \Bigr) \\
\end{array}
\right.  \qquad (11) \; (\leftarrow 1.2 式(18))
$$

と求まる。